Trail and Error Method

I used this notebook for identifing the structure of website, API call etc

```
First trying to print all the grama panjayaths of malappuram
```



In [ ]:
import requests
import pandas as pd
import time
import urllib3

# Disable SSL warnings (common for these gov sites)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- CORRECTED URLS  ---
BASE_URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
BASE_URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

# Mapping: G=Grama Pyt, B=Block, D=District, M=Municipality, C=Corp
LB_TYPES = {
    "G": "Grama Panchayat",
    # Uncomment these when ready to scrape everything
    # "B": "Block Panchayat",
    # "D": "District Panchayat",
    # "M": "Municipality",
    # "C": "Corporation"
}

def scrape_kerala_election(year=2020):
    master_data = []

    # Testing with just District 1 (Thiruvananthapuram) first
    districts = [1]

    print(f"--- Starting Scraping for Year {year} ---")

    with requests.Session() as session:
        session.headers.update(HEADERS)

        for dist_id in districts:
            for lb_code, lb_name in LB_TYPES.items():

                # 1. GET LIST OF LOCAL BODIES
                payload_list = {"year": year, "district": dist_id, "type": lb_code}

                try:
                    print(f"Fetching list for Dist {dist_id} - {lb_name}...")
                    resp = session.post(BASE_URL_LIST, data=payload_list, verify=False)

                    if resp.status_code != 200:
                        print(f"Error: Status {resp.status_code}")
                        continue

                    data_list = resp.json()
                    # The list is in "summary" based on your previous logs
                    local_bodies = data_list.get("summary", [])

                    print(f"  -> Found {len(local_bodies)} local bodies.")

                    # 2. GET DETAILS FOR EACH BODY
                    # Limit to first 3 bodies for testing
                    for body in local_bodies[:3]:
                        lb_id = body[0]   # ID (e.g. G001)
                        lb_text = body[1] # Name (e.g. Adimaly)

                        # PAYLOAD FOR DETAILS
                        # NOTE: Ensure key is 'lb_id' or 'lbid' based on your network tab!
                        payload_details = {"year": year, "lb_id": lb_id}

                        resp_det = session.post(BASE_URL_DETAILS, data=payload_details, verify=False)
                        det_json = resp_det.json()

                        # FIND THE WARDS DATA
                        # It is likely under 'wards', 'data', or just the root list.
                        # We try to find the list dynamically:
                        wards = []
                        if "wards" in det_json: wards = det_json["wards"]
                        elif "data" in det_json: wards = det_json["data"]
                        elif isinstance(det_json, list): wards = det_json

                        for w in wards:
                            # Mapping based on your table screenshot:
                            # Col 0: Party (UDF)
                            # Col 1: Ward No (001)
                            # Col 2: Ward Name (Valliyaparamba)
                            # Col 4: Candidate Name (Malayalam text)
                            # Col 5: Votes

                            master_data.append({
                                "District": dist_id,
                                "Local Body": lb_text,
                                "Type": lb_name,
                                "WardName": f"{w[1]} - {w[2]}",
                                "Candidate": w[4],
                                "Party": w[0],
                                "Front": w[0], # Usually same as party in this summary
                                "Votes": w[5],
                                "Year": year
                            })

                        time.sleep(0.5) # Be polite

                except Exception as e:
                    print(f"  -> Error: {e}")

    # SAVE
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING SUCCESS! ---")
        print(df.head())
        df.to_csv("kerala_election_sample.csv", index=False)
    else:
        print("No data found. Check 'lb_id' payload key.")

if __name__ == "__main__":
    scrape_kerala_election()

--- Starting Scraping for Year 2020 ---
Fetching list for Dist 1 - Grama Panchayat...
  -> Error: Expecting value: line 8 column 1 (char 7)
No data found. Check 'lb_id' payload key.


In [ ]:
master_data

[]

In [ ]:
import requests
import urllib3

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# CONFIG
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def debug_response():
    print("--- DEBUGGING SERVER RESPONSE ---")

    # We try 'd' instead of 'district' just in case that's the issue
    # We also try 't' instead of 'type'
    payload = {
        "year": 2020,
        "d": 1,        # Common abbreviation in PHP sites
        "district": 1, # Sending both to be safe
        "type": "G",
        "t": "G"
    }

    try:
        resp = requests.post(URL_LIST, data=payload, headers=HEADERS, verify=False)

        print(f"Status Code: {resp.status_code}")
        print("\n--- RAW RESPONSE TEXT (First 500 chars) ---")
        # This will show us the PHP error or the whitespace
        print(resp.text[:500])
        print("-------------------------------------------")

        # Check if it works if we strip whitespace
        clean_text = resp.text.strip()
        if clean_text.startswith(("<br", "<b", "Error")):
            print("DIAGNOSIS: Server returned an HTML Error (likely PHP error).")
        elif not clean_text:
            print("DIAGNOSIS: Server returned EMPTY response.")
        else:
            print("DIAGNOSIS: Response received. Trying to parse...")
            try:
                data = resp.json()
                print("SUCCESS! JSON Parsed.")
                print("Keys found:", data.keys())
            except:
                print("JSON Parse Failed. The data might be in a different format (e.g., JS Array).")

    except Exception as e:
        print(f"Connection Error: {e}")

if __name__ == "__main__":
    debug_response()

--- DEBUGGING SERVER RESPONSE ---
Status Code: 200

--- RAW RESPONSE TEXT (First 500 chars) ---








-------------------------------------------
DIAGNOSIS: Server returned EMPTY response.


In [ ]:
import requests
import urllib3
import time

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# CONFIG
HOME_URL = "https://lbtrend.kerala.gov.in/"
AJAX_URL = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Origin": "https://lbtrend.kerala.gov.in",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def find_correct_payload():
    print("--- 1. Getting Session Cookies ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    try:
        # Visit homepage to get PHPSESSID
        session.get(HOME_URL, verify=False)
        print("Cookies obtained:", session.cookies.get_dict())
    except Exception as e:
        print(f"Could not load homepage: {e}")

    print("\n--- 2. Brute-Forcing Parameter Names ---")

    # We know the VALUES (Malappuram=10, or TVM=1, Grama Panchayat=G)
    # We just need the KEYS.

    # Common variations for "District"
    dist_keys = ["district", "dist", "d", "dist_id", "district_id", "districtId", "did"]
    # Common variations for "Type"
    type_keys = ["type", "lb_type", "t", "lbt", "body_type", "lbtype"]

    found = False

    for d_key in dist_keys:
        for t_key in type_keys:
            if found: break

            # Construct payload
            payload = {
                "year": 2020,
                d_key: 1,  # Thiruvananthapuram
                t_key: "G" # Grama Panchayat
            }

            try:
                # print(f"Trying keys: {d_key}, {t_key} ...", end="\r")
                resp = session.post(AJAX_URL, data=payload, verify=False)

                # Check if we got JSON
                if resp.status_code == 200 and len(resp.text.strip()) > 0:
                    try:
                        data = resp.json()
                        # If we get a dictionary with "summary" or "mdata", we won!
                        if "summary" in data or "mdata" in data:
                            print(f"\n\n[SUCCESS] FOUND THE KEYS!")
                            print(f"Correct District Key: '{d_key}'")
                            print(f"Correct Type Key:     '{t_key}'")
                            print(f"Response snippet: {str(data)[:100]}...")
                            found = True
                    except:
                        pass # Not JSON

            except Exception:
                pass

    if not found:
        print("\n\n[FAILURE] None of the combinations worked.")
        print("Please manually check the 'Payload' tab in your browser Network tools.")
        print("Look for the 'Form Data' section.")

if __name__ == "__main__":
    find_correct_payload()

--- 1. Getting Session Cookies ---
Cookies obtained: {}

--- 2. Brute-Forcing Parameter Names ---


[FAILURE] None of the combinations worked.
Please manually check the 'Payload' tab in your browser Network tools.
Look for the 'Form Data' section.


In [ ]:
import requests
import pandas as pd
import time
import urllib3

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- CONFIGURATION ---
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def scrape_malappuram():
    print("--- Starting Scraper for Malappuram (D10001) ---")

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. GET THE LIST OF PANCHAYATS
    # Using the exact keys you found in the payload tab
    payload_list = {
        "_p": "dv",      # Fixed value
        "_l": "P",       # Fixed value (Likely 'P' for Grama Panchayat)
        "_d": "D10001",  # Malappuram District Code
        "_s": "L"        # Fixed value
    }

    try:
        print("1. Fetching list of Local Bodies...")
        resp_list = session.post(URL_LIST, data=payload_list, verify=False)

        # Check for errors
        if resp_list.status_code != 200:
            print(f"Error: Status {resp_list.status_code}")
            return

        # Parse JSON
        data_list = resp_list.json()

        # In your screenshot, the list was inside 'summary' or 'mdata'
        # Adjust this key if needed based on the response you get
        local_bodies = []
        if "summary" in data_list:
            local_bodies = data_list["summary"]
        elif "mdata" in data_list:
            # Sometimes it's inside mdata, we'll print keys to be sure
             print("Keys found in list:", data_list.keys())
             # Fallback: try to find a list anywhere
             for k, v in data_list.items():
                 if isinstance(v, list):
                     local_bodies = v
                     break

        print(f"   -> Found {len(local_bodies)} Local Bodies.")

        # 2. LOOP THROUGH EACH LOCAL BODY
        master_data = []

        # We limit to 5 bodies for this test to be fast
        for body in local_bodies[:5]:
            lb_id = body[0]   # e.g., "G10072"
            lb_name = body[1] # e.g., "Abdurahman Nagar"

            print(f"2. Fetching details for {lb_name} ({lb_id})...")

            # PAYLOAD FOR DETAILS
            # We assume the key is 'lb_id' or 'id'.
            # If this fails, check the Payload tab for lb_ajax2.php!
            payload_details = {
                "lb_id": lb_id,
                "year": 2020    # Often required
            }

            resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)

            if resp_det.status_code == 200:
                det_json = resp_det.json()

                # EXTRACT WARD DATA
                # Look for 'wards', 'data', or just the root list
                wards = []
                if "wards" in det_json: wards = det_json["wards"]
                elif "data" in det_json: wards = det_json["data"]

                for w in wards:
                    # Mapping based on your screenshot:
                    # Col 1: Ward No, Col 2: Ward Name, Col 4: Candidate, Col 5: Votes
                    master_data.append({
                        "District": "Malappuram",
                        "Local Body": lb_name,
                        "Ward": f"{w[1]} - {w[2]}",
                        "Candidate": w[4],
                        "Party": w[0],
                        "Votes": w[5]
                    })

            time.sleep(0.5) # Be polite

        # 3. SAVE RESULTS
        if master_data:
            df = pd.DataFrame(master_data)
            print("\n--- SUCCESS! Sample Data ---")
            print(df.head())
            df.to_csv("Malappuram_Test.csv", index=False)
        else:
            print("List fetched, but no ward details found. Check 'lb_id' key.")

    except Exception as e:
        print(f"Critical Error: {e}")

if __name__ == "__main__":
    scrape_malappuram()

--- Starting Scraper for Malappuram (D10001) ---
1. Fetching list of Local Bodies...
   -> Found 5 Local Bodies.
2. Fetching details for MALAPPURAM (D10001)...
Critical Error: Expecting value: line 1 column 1 (char 0)


In [ ]:
import requests
import pandas as pd
import time
import urllib3

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

# 2. DEFINITIONS
# Code mapping for the 5 types of bodies
# 'P' seems to be the code for Grama Panchayat based on your screenshot
LB_TYPES = {
    "P": "Grama Panchayat",
    # "B": "Block Panchayat", # Enable these later
    # "D": "District Panchayat",
    # "M": "Municipality",
    # "C": "Corporation"
}

def get_district_code(number):
    # Formats number 1 to "D10001", 10 to "D10010"
    return f"D100{number:02d}"

def scrape_kerala_election():
    print("--- Starting Final Scraper ---")
    master_data = []

    # Session keeps our connection alive
    session = requests.Session()
    session.headers.update(HEADERS)

    # Loop through Districts (Malappuram is usually 10, but let's try 1 to 14)
    # For testing, we just do ONE district. Change [10] to range(1, 15) for all.
    for dist_num in [10]:
        dist_code = get_district_code(dist_num) # e.g., D10010
        print(f"\nProcessing District: {dist_code}")

        for type_code, type_name in LB_TYPES.items():

            # --- STEP 1: GET LIST OF LOCAL BODIES ---
            payload_list = {
                "_p": "dv",       # Constant
                "_l": type_code,  # 'P' for Panchayat
                "_d": dist_code,  # 'D10010'
                "_s": "L"         # Constant
            }

            try:
                resp = session.post(URL_LIST, data=payload_list, verify=False)
                data_list = resp.json()

                # Extract the list from 'summary' or 'mdata'
                # Based on your logs, it's a list inside 'summary'
                local_bodies = data_list.get("summary", [])

                if not local_bodies:
                    print(f"  No {type_name} found for {dist_code}. Checking 'mdata'...")
                    local_bodies = data_list.get("mdata", [])

                print(f"  -> Found {len(local_bodies)} {type_name}s. Scraping details...")

                # --- STEP 2: LOOP THROUGH EACH BODY ---
                # We scrape the first 5 for testing. Remove [:5] to scrape ALL.
                for item in local_bodies[:5]:
                    lb_id = item[0]    # e.g., G10072
                    lb_name = item[1]  # e.g., Abdurehman Nagar

                    # PAYLOAD FOR DETAILS (The keys you found!)
                    payload_details = {
                        "_p": "wv",       # Ward View
                        "_w": lb_id,      # The ID (G10072)
                        "_t": type_code,  # Type (P)
                        "_s": "L"         # Constant
                    }

                    resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)
                    data_det = resp_det.json()

                    # Extract Wards
                    # Based on your screenshot, data is likely in 'mdata' -> 'wards' or just 'mdata'
                    wards = []
                    mdata = data_det.get("mdata", {})

                    # Handle different JSON structures safely
                    if isinstance(mdata, dict):
                         # If mdata is a dict, look for 'wards' or 'data' inside
                         if "wards" in mdata: wards = mdata["wards"]
                         elif "data" in mdata: wards = mdata["data"]
                    elif isinstance(mdata, list):
                         # If mdata is directly the list
                         wards = mdata

                    # If still empty, check root level
                    if not wards and "wards" in data_det:
                        wards = data_det["wards"]

                    # PARSE THE ROWS
                    for w in wards:
                        # Index Mapping (Based on your table screenshot):
                        # 0: Party (UDF) | 1: Ward No | 2: Ward Name | 4: Candidate | 5: Votes
                        master_data.append({
                            "District": dist_code,
                            "Local Body": lb_name,
                            "Type": type_name,
                            "Ward": f"{w[1]} - {w[2]}",
                            "Candidate": w[4],
                            "Party": w[0],
                            "Votes": w[5],
                            "Year": 2020 # You can loop years too
                        })

                    # Polite delay
                    time.sleep(0.2)

            except Exception as e:
                print(f"  Error on {dist_code} - {type_name}: {e}")

    # 3. SAVE
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SUCCESS! Sample Data ---")
        print(df.head())
        df.to_csv("Final_Kerala_Election_Data.csv", index=False)
        print(f"Saved {len(df)} rows to Final_Kerala_Election_Data.csv")
    else:
        print("Scraping finished but no data found. Check Dist Code or JSON keys.")

if __name__ == "__main__":
    scrape_kerala_election()

--- Starting Final Scraper ---

Processing District: D10010
  Error on D10010 - Grama Panchayat: Expecting value: line 8 column 1 (char 7)
Scraping finished but no data found. Check Dist Code or JSON keys.


In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def scrape_malappuram_final():
    print("--- Starting Scraper (Target: Malappuram / D10001) ---")
    master_data = []

    # 2. START SESSION (Crucial for PHP Sites)
    session = requests.Session()
    session.headers.update(HEADERS)

    try:
        # Visit homepage first to get the Session Cookie
        session.get(URL_HOME, verify=False)
        print("Cookies initialized.")
    except Exception as e:
        print(f"Warning: Could not fetch homepage ({e}). Proceeding anyway...")

    # 3. STEP 1: GET LIST (Using D10001 from your screenshot)
    payload_list = {
        "_p": "dv",
        "_l": "P",        # Grama Panchayat
        "_d": "D10001",   # <--- HARDCODED FROM YOUR SCREENSHOT
        "_s": "L"
    }

    try:
        print(f"Fetching list for District Code: {payload_list['_d']}...")
        resp = session.post(URL_LIST, data=payload_list, verify=False)

        # DEBUG: If this fails, we print the raw text
        if resp.status_code != 200:
            print(f"Server Error: Status {resp.status_code}")
            return

        try:
            data_list = resp.json()
        except json.JSONDecodeError:
            print("CRITICAL ERROR: Server did not return JSON.")
            print(f"Raw Response: {resp.text[:500]}") # Show first 500 chars
            return

        # Find the list inside the JSON
        # It usually hides in 'summary' or 'mdata'
        local_bodies = data_list.get("summary", [])
        if not local_bodies:
            local_bodies = data_list.get("mdata", [])

        print(f"-> Found {len(local_bodies)} Local Bodies.")

        # 4. STEP 2: LOOP DETAILS
        # Scrape first 10 for testing
        for item in local_bodies[:10]:
            lb_id = item[0]    # e.g., G10072
            lb_name = item[1]  # e.g., Abdurehman Nagar

            # Payload for 'lb_ajax2.php' (From your screenshot)
            payload_details = {
                "_p": "wv",       # Ward View
                "_w": lb_id,      # The ID (G10072)
                "_t": "P",        # Type
                "_s": "L"
            }

            # Request
            resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)

            try:
                data_det = resp_det.json()

                # Extract Wards (Handling variations in structure)
                wards = []
                # Check 'mdata' -> 'wards'
                if "mdata" in data_det and isinstance(data_det["mdata"], dict):
                    wards = data_det["mdata"].get("wards", [])
                # Check 'mdata' directly if it's a list
                elif "mdata" in data_det and isinstance(data_det["mdata"], list):
                    wards = data_det["mdata"]
                # Check root 'wards'
                elif "wards" in data_det:
                    wards = data_det["wards"]

                # Add to Master Data
                for w in wards:
                    # Column Mapping based on your table
                    master_data.append({
                        "District": "Malappuram",
                        "Local Body": lb_name,
                        "Type": "Grama Panchayat",
                        "Ward": f"{w[1]} - {w[2]}", # Ward No + Name
                        "Candidate": w[4],
                        "Party": w[0],
                        "Votes": w[5],
                        "Year": 2025
                    })

            except Exception as e:
                print(f"Error parsing {lb_name}: {e}")

            time.sleep(0.1) # Be polite

    except Exception as e:
        print(f"Global Error: {e}")

    # 5. SAVE
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SUCCESS! Sample Data ---")
        print(df.head())
        df.to_csv("Malappuram_Result_Fixed.csv", index=False)
        print(f"Saved {len(df)} rows to CSV.")
    else:
        print("Scraping finished but no data found.")

if __name__ == "__main__":
    scrape_malappuram_final()

--- Starting Scraper (Target: Malappuram / D10001) ---
Cookies initialized.
Fetching list for District Code: D10001...
-> Found 5 Local Bodies.
Scraping finished but no data found.


INSPECTING JSON STRUCTURE

In [ ]:
import requests
import urllib3
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# CONFIG
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def inspect_structure():
    print("--- INSPECTING JSON STRUCTURE ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. FETCH THE LIST (District 10 - Malappuram)
    payload_list = {
        "_p": "dv",
        "_l": "P",
        "_d": "D10001",
        "_s": "L"
    }

    try:
        print("Fetching List for D10001...")
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data = resp.json()

        print(f"\n[ROOT KEYS]: {list(data.keys())}")

        # Check 'mdata' contents if it exists
        if "mdata" in data:
            mdata = data["mdata"]
            if isinstance(mdata, dict):
                print(f"[mdata KEYS]: {list(mdata.keys())}")

                # Check likely candidates for the big list
                for k in mdata.keys():
                    val = mdata[k]
                    if isinstance(val, list):
                        print(f"  -> Key '{k}' contains a LIST of length {len(val)}")
                        if len(val) > 0:
                            print(f"     Sample Item: {str(val[0])[:100]}...")
            elif isinstance(mdata, list):
                print(f"[mdata IS A LIST] Length: {len(mdata)}")
                if len(mdata) > 0:
                    print(f"  Sample Item: {str(mdata[0])[:100]}...")

        # Check 'summary' just to confirm it's the wrong one
        if "summary" in data:
            print(f"\n[summary] Length: {len(data['summary'])} (Expected ~5 rows for party stats)")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    inspect_structure()

--- INSPECTING JSON STRUCTURE ---
Fetching List for D10001...

[ROOT KEYS]: ['mdata', 'total', 'summary', 'payload']
[mdata KEYS]: ['rls', 'info', 'ut']

[summary] Length: 5 (Expected ~5 rows for party stats)


Auto-Detecting Correct Parameters

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def scrape_with_key_search():
    print("--- 1. Auto-Detecting Correct Parameters ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    # Get Session Cookie
    try:
        session.get(URL_HOME, verify=False)
    except:
        pass

    # We need to find the '_s' value that gives us a list of 90+ panchayats
    # 'L' gave us 5 rows (Summary). Let's try others.
    # Common guesses: M=Majority List, S=Status List, O=Other, D=Details
    candidate_keys = ["M", "S", "W", "O", "D", "T", "L"]

    correct_s_key = None
    local_bodies_found = []

    for s_val in candidate_keys:
        payload_test = {
            "_p": "dv",
            "_l": "P",
            "_d": "D10001",  # Malappuram
            "_s": s_val      # Testing this!
        }

        try:
            resp = session.post(URL_LIST, data=payload_test, verify=False)
            data = resp.json()

            # The list might be in different places depending on the view
            # We look for ANY list longer than 10 items
            current_list = []

            # Check root keys for a list
            for key, val in data.items():
                if isinstance(val, list) and len(val) > 10:
                    current_list = val
                    break

            # Check inside 'mdata'
            if not current_list and "mdata" in data and isinstance(data["mdata"], dict):
                for key, val in data["mdata"].items():
                    if isinstance(val, list) and len(val) > 10:
                        current_list = val
                        break

            # Check inside 'summary'
            if not current_list and "summary" in data and len(data["summary"]) > 10:
                current_list = data["summary"]

            print(f"Trying _s='{s_val}' -> Found {len(current_list) if current_list else 0} items.")

            if len(current_list) > 10:
                print(f"--> MATCH! '{s_val}' is the correct key.")
                correct_s_key = s_val
                local_bodies_found = current_list
                break

        except Exception as e:
            pass

    if not correct_s_key:
        print("\n[ERROR] Could not find the correct parameter. The server might require a specific sequence.")
        return

    # --- 2. SCRAPING DETAILS ---
    print(f"\n--- 2. Scraping Details using _s='{correct_s_key}' ---")
    master_data = []

    # Scrape first 5 bodies for testing
    for item in local_bodies_found[:5]:
        # Item format is likely: ["G10072", "Abdurehman Nagar", ...] or similar
        # We need to be careful with indices.
        # Usually ID is index 0 or 1.

        # Let's verify if item[0] looks like an ID (starts with G)
        lb_id = item[0]
        lb_name = item[1]

        # If ID doesn't start with G, check other columns (sometimes order changes)
        if not str(lb_id).startswith("G"):
            # Fallback search for ID
            for col in item:
                if str(col).startswith("G1"):
                    lb_id = col
                    break

        print(f"Fetching: {lb_name} ({lb_id})")

        # Payload for Details
        payload_det = {
            "_p": "wv",
            "_w": lb_id,
            "_t": "P",
            "_s": "L"  # This seems constant for details based on your screenshot
        }

        try:
            resp_det = session.post(URL_DETAILS, data=payload_det, verify=False)
            det_json = resp_det.json()

            # Extract Wards
            wards = []
            if "mdata" in det_json and isinstance(det_json["mdata"], dict):
                # Try common keys inside mdata
                for k in ["wards", "data", "rows"]:
                    if k in det_json["mdata"]:
                        wards = det_json["mdata"][k]
                        break
            elif "wards" in det_json:
                wards = det_json["wards"]

            for w in wards:
                master_data.append({
                    "District": "Malappuram",
                    "Local Body": lb_name,
                    "Ward": f"{w[1]} - {w[2]}",
                    "Candidate": w[4],
                    "Party": w[0],
                    "Votes": w[5],
                    "Year": 2020
                })

        except Exception as e:
            print(f"Error on {lb_name}: {e}")

        time.sleep(0.1)

    # --- 3. SAVE ---
    if master_data:
        df = pd.DataFrame(master_data)
        print("\nSUCCESS!")
        print(df.head())
        df.to_csv("Final_Kerala_Data.csv", index=False)
    else:
        print("No detail data found.")

if __name__ == "__main__":
    scrape_with_key_search()

--- 1. Auto-Detecting Correct Parameters ---
Trying _s='M' -> Found 82 items.
--> MATCH! 'M' is the correct key.

--- 2. Scraping Details using _s='M' ---
Fetching: Abdurehman Nagar (G10072)
Fetching: Alamkode (G10096)
Fetching: Aliparamba (G10043)
Fetching: Amarambalam (G10026)
Fetching: Anakayam (G10037)
No detail data found.


INSPECTING WARD DETAILS JSON

In [ ]:
import requests
import urllib3
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# URLs
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def inspect_details_structure():
    print("--- INSPECTING WARD DETAILS JSON ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    # We use the ID for 'Abdurehman Nagar' which we found earlier: G10072
    payload = {
        "_p": "wv",
        "_w": "G10072",
        "_t": "P",
        "_s": "L"
    }

    try:
        print("Fetching details for G10072...")
        resp = session.post(URL_DETAILS, data=payload, verify=False)
        data = resp.json()

        # 1. Print Root Keys
        print(f"\n[ROOT KEYS]: {list(data.keys())}")

        # 2. Inspect 'mdata'
        if "mdata" in data:
            mdata = data["mdata"]
            if isinstance(mdata, dict):
                print(f"[mdata KEYS]: {list(mdata.keys())}")

                # Check for lists inside mdata
                for k, v in mdata.items():
                    if isinstance(v, list):
                        print(f"  -> Found LIST in key '{k}' (Length: {len(v)})")
                        if len(v) > 0:
                            print(f"     Sample Row: {v[0]}")

            elif isinstance(mdata, list):
                print(f"  -> 'mdata' IS the list! (Length: {len(mdata)})")
                if len(mdata) > 0:
                    print(f"     Sample Row: {mdata[0]}")

    except Exception as e:
        print(f"Error: {e}")
        # If JSON fails, print raw text
        print("Raw Response:", resp.text[:200])

if __name__ == "__main__":
    inspect_details_structure()

--- INSPECTING WARD DETAILS JSON ---
Fetching details for G10072...

[ROOT KEYS]: ['mdata', 'payload', 'summary']
[mdata KEYS]: ['rls', 'info', 'ut']


Starting Final Scraper ( failed )

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def scrape_kerala_final():
    print("--- Starting Final Scraper ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. CRITICAL: GET SESSION COOKIE
    try:
        print("1. Initializing Session (Visiting Homepage)...")
        session.get(URL_HOME, verify=False)
    except Exception as e:
        print(f"   Warning: Homepage visit failed ({e}). trying anyway...")

    # MALAPPURAM CONFIG (Hardcoded based on your success)
    DISTRICT_CODE = "D10001"
    DISTRICT_NAME = "Malappuram"

    # --- STEP 1: GET LIST OF PANCHAYATS ---
    print(f"2. Fetching Panchayat List for {DISTRICT_NAME} ({DISTRICT_CODE})...")

    payload_list = {
        "_p": "dv",
        "_l": "P",
        "_d": DISTRICT_CODE,
        "_s": "M"         # "M" = Majority List (Verified)
    }

    try:
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data_list = resp.json()

        # AGGRESSIVE LIST FINDER
        # We look for ANY list with > 10 items in 'summary', 'mdata', or root
        local_bodies = []

        # Check 'summary' (Most likely location for _s='M')
        if "summary" in data_list and isinstance(data_list["summary"], list) and len(data_list["summary"]) > 10:
            local_bodies = data_list["summary"]

        # Fallback: Check 'mdata'
        elif "mdata" in data_list:
            if isinstance(data_list["mdata"], list) and len(data_list["mdata"]) > 10:
                local_bodies = data_list["mdata"]
            elif isinstance(data_list["mdata"], dict):
                for k, v in data_list["mdata"].items():
                    if isinstance(v, list) and len(v) > 10:
                        local_bodies = v
                        break

        if not local_bodies:
            print("   [ERROR] List is empty. Server response keys:", data_list.keys())
            if "summary" in data_list: print("   Summary length:", len(data_list["summary"]))
            return

        print(f"   -> Found {len(local_bodies)} Local Bodies.")

        # --- STEP 2: LOOP THROUGH EACH LOCAL BODY ---
        print("3. Scraping Ward Details...")

        # Remove [:5] to scrape ALL 90+ bodies. Keeping it for test speed.
        # Change to `for item in local_bodies:` for full run.
        for item in local_bodies:
            # Item format check
            lb_id = item[0]
            lb_name = item[1]

            # Robust ID extraction
            if not str(lb_id).startswith("G"):
                for col in item:
                    if str(col).startswith("G1"):
                        lb_id = col
                        break

            print(f"   -> Processing: {lb_name} ({lb_id})")

            # Payload for Details (Verified keys)
            payload_details = {
                "_p": "wv",
                "_w": lb_id,
                "_t": "P",
                "_s": "L"
            }

            try:
                resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)
                det_json = resp_det.json()

                # EXTRACT WARDS
                # Verified: Data is in 'summary' for this view
                wards = []
                if "summary" in det_json:
                    wards = det_json["summary"]
                elif "mdata" in det_json:
                    # Fallback check
                    if isinstance(det_json["mdata"], list): wards = det_json["mdata"]
                    elif isinstance(det_json["mdata"], dict) and "wards" in det_json["mdata"]:
                        wards = det_json["mdata"]["wards"]

                if not wards:
                    print(f"      [Warning] No wards found for {lb_name}")

                # Add to Master Data
                for w in wards:
                    master_data.append({
                        "District": DISTRICT_NAME,
                        "Local Body": lb_name,
                        "Type": "Grama Panchayat",
                        "WardName": f"{w[1]} - {w[2]}",
                        "Candidate": w[4],
                        "Party": w[0],
                        "Front": w[0],
                        "Votes": w[5],
                        "Year": 2020
                    })

            except Exception as e:
                print(f"      Error: {e}")

            time.sleep(0.1)

    except Exception as e:
        print(f"Global Error: {e}")

    # --- 3. SAVE TO CSV ---
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING COMPLETE ---")
        print(df.head())
        filename = "Malappuram_Master_Dataset.csv"
        df.to_csv(filename, index=False)
        print(f"Saved {len(df)} rows to {filename}")
    else:
        print("No data collected.")

if __name__ == "__main__":
    scrape_kerala_final()

--- Starting Final Scraper ---
1. Initializing Session (Visiting Homepage)...
2. Fetching Panchayat List for Malappuram (D10001)...
   [ERROR] List is empty. Server response keys: dict_keys(['mdata', 'total', 'summary', 'payload'])
   Summary length: 5


Starting Final Scraper (Target: Malappuram 2020) Again failed

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def scrape_kerala_final():
    print("--- Starting Final Scraper (Target: Malappuram 2020) ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. VISITING HOMEPAGE (Gets the Session Cookie)
    try:
        session.get(URL_HOME, verify=False)
        print("1. Session initialized.")
    except Exception as e:
        print(f"   Warning: Session init failed ({e})")

    # CONFIGURATION
    DISTRICT_CODE = "D10001"
    DISTRICT_NAME = "Malappuram"
    TARGET_YEAR = 2020

    # --- STEP 1: GET LIST OF PANCHAYATS ---
    print(f"2. Fetching Panchayat List...")

    payload_list = {
        "_p": "dv",
        "_l": "P",
        "_d": DISTRICT_CODE,
        "_s": "M",        # Majority View (List of Bodies)
        "_y": TARGET_YEAR # <--- ADDED YEAR (Crucial fix)
    }

    try:
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data_list = resp.json()

        # AGGRESSIVE LIST FINDER
        local_bodies = []

        # Check 'mdata' for the list (Common for Majority View)
        if "mdata" in data_list:
            # If mdata IS the list
            if isinstance(data_list["mdata"], list) and len(data_list["mdata"]) > 10:
                local_bodies = data_list["mdata"]
            # If mdata is a dict containing the list (e.g., in 'rows' or 'data')
            elif isinstance(data_list["mdata"], dict):
                for k, v in data_list["mdata"].items():
                    if isinstance(v, list) and len(v) > 10:
                        local_bodies = v
                        break

        # Fallback to 'summary'
        if not local_bodies and "summary" in data_list and len(data_list["summary"]) > 10:
            local_bodies = data_list["summary"]

        if not local_bodies:
            print("   [ERROR] List is empty.")
            print("   Server returned keys:", data_list.keys())
            if "mdata" in data_list and isinstance(data_list["mdata"], dict):
                print("   mdata keys:", data_list["mdata"].keys())
            return

        print(f"   -> Found {len(local_bodies)} Local Bodies.")

        # --- STEP 2: LOOP THROUGH EACH LOCAL BODY ---
        print("3. Scraping Ward Details...")

        for item in local_bodies:
            # ROBUST ID EXTRACTION
            # Sometimes ID is at index 0, sometimes 1. We find the "G..." code.
            lb_id = None
            lb_name = "Unknown"

            for col in item:
                s_col = str(col)
                if s_col.startswith("G1"): # G1xxxxx is the ID format
                    lb_id = s_col
                elif len(s_col) > 4 and not s_col.startswith("G") and not s_col.isdigit():
                    lb_name = s_col # Heuristic for Name

            # Fallback if heuristics fail (use fixed indices)
            if not lb_id: lb_id = item[0]
            if lb_name == "Unknown": lb_name = item[1]

            print(f"   -> Processing: {lb_name} ({lb_id})")

            # PAYLOAD FOR DETAILS
            payload_details = {
                "_p": "wv",
                "_w": lb_id,
                "_t": "P",
                "_s": "L",
                "_y": TARGET_YEAR # <--- ADDED YEAR
            }

            try:
                resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)
                det_json = resp_det.json()

                # EXTRACT WARDS
                wards = []
                # Check summary first (Verified location for Ward List)
                if "summary" in det_json and len(det_json["summary"]) > 0:
                    wards = det_json["summary"]
                elif "mdata" in det_json:
                    if isinstance(det_json["mdata"], list): wards = det_json["mdata"]
                    elif isinstance(det_json["mdata"], dict):
                         # Try to find list inside mdata dict
                         for k,v in det_json["mdata"].items():
                             if isinstance(v, list): wards = v; break

                if not wards:
                    print(f"      [Warning] No wards found.")

                # Add to Master Data
                for w in wards:
                    # w format: [Party, WardNo, WardName, ..., Candidate, Votes]
                    # Adjust indices if your CSV looks wrong
                    master_data.append({
                        "District": DISTRICT_NAME,
                        "Local Body": lb_name,
                        "Type": "Grama Panchayat",
                        "WardName": f"{w[1]} - {w[2]}",
                        "Candidate": w[4],
                        "Party": w[0],
                        "Front": w[0],
                        "Votes": w[5],
                        "Year": TARGET_YEAR
                    })

            except Exception as e:
                print(f"      Error: {e}")

            time.sleep(0.1) # Be polite

    except Exception as e:
        print(f"Global Error: {e}")

    # --- 3. SAVE TO CSV ---
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING COMPLETE ---")
        print(df.head())
        filename = f"Malappuram_{TARGET_YEAR}_Data.csv"
        df.to_csv(filename, index=False)
        print(f"Saved {len(df)} rows to {filename}")
    else:
        print("No data collected.")

if __name__ == "__main__":
    scrape_kerala_final()

--- Starting Final Scraper (Target: Malappuram 2020) ---
1. Session initialized.
2. Fetching Panchayat List...
   [ERROR] List is empty.
   Server returned keys: dict_keys(['mdata', 'total', 'summary', 'payload'])
   mdata keys: dict_keys(['rls', 'info', 'ut'])


In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def scrape_kerala_final():
    print("--- Starting Final Scraper (Target: Malappuram) ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. VISITING HOMEPAGE (Essential for Session Cookies)
    try:
        session.get(URL_HOME, verify=False)
        print("1. Session initialized.")
    except Exception as e:
        print(f"   Warning: Session init failed ({e})")

    # CONFIGURATION
    DISTRICT_CODE = "D10001"
    DISTRICT_NAME = "Malappuram"

    # --- STEP 1: GET LIST OF PANCHAYATS ---
    print(f"2. Fetching Panchayat List...")

    # REVERTED PAYLOAD: Removed '_y' because it caused the error.
    # Using exactly what worked in your Auto-Detect run.
    payload_list = {
        "_p": "dv",
        "_l": "P",
        "_d": DISTRICT_CODE,
        "_s": "M"  # "M" = Majority List (Verified to return ~82 items)
    }

    try:
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data_list = resp.json()

        local_bodies = []

        # 1. Check 'mdata' for the list (Common for Majority View)
        if "mdata" in data_list:
            if isinstance(data_list["mdata"], list) and len(data_list["mdata"]) > 10:
                local_bodies = data_list["mdata"]
            elif isinstance(data_list["mdata"], dict):
                for k, v in data_list["mdata"].items():
                    if isinstance(v, list) and len(v) > 10:
                        local_bodies = v
                        break

        # 2. Check 'summary' (Fallback)
        if not local_bodies and "summary" in data_list and len(data_list["summary"]) > 10:
            local_bodies = data_list["summary"]

        if not local_bodies:
            print("   [ERROR] List is empty.")
            print("   The server might have reverted to default view.")
            return

        print(f"   -> Found {len(local_bodies)} Local Bodies.")

        # --- STEP 2: LOOP THROUGH EACH LOCAL BODY ---
        print("3. Scraping Ward Details...")

        # We loop through ALL found bodies
        for item in local_bodies:
            # ROBUST ID EXTRACTION
            lb_id = None
            lb_name = "Unknown"

            # Find the ID (starts with "G1")
            for col in item:
                s_col = str(col)
                if s_col.startswith("G1"):
                    lb_id = s_col
                elif len(s_col) > 3 and not s_col.startswith("G") and not s_col.isdigit():
                    lb_name = s_col

            # Fallback
            if not lb_id: lb_id = item[0]
            if lb_name == "Unknown": lb_name = item[1]

            print(f"   -> Processing: {lb_name} ({lb_id})")

            # PAYLOAD FOR DETAILS
            # Verified in your Inspector run
            payload_details = {
                "_p": "wv",
                "_w": lb_id,
                "_t": "P",
                "_s": "L"
            }

            try:
                resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)
                det_json = resp_det.json()

                # EXTRACT WARDS
                wards = []

                # Verified: Data is often in 'summary' for this view
                if "summary" in det_json and len(det_json["summary"]) > 0:
                    wards = det_json["summary"]
                elif "mdata" in det_json:
                    if isinstance(det_json["mdata"], list): wards = det_json["mdata"]
                    elif isinstance(det_json["mdata"], dict):
                         for k,v in det_json["mdata"].items():
                             if isinstance(v, list): wards = v; break

                if not wards:
                    print(f"      [Warning] No wards found.")

                # Add to Master Data
                for w in wards:
                    # w format usually: [Party, WardNo, WardName, ..., Candidate, Votes]
                    # Adjust indices based on your specific CSV output if needed
                    master_data.append({
                        "District": DISTRICT_NAME,
                        "Local Body": lb_name,
                        "Type": "Grama Panchayat",
                        "WardName": f"{w[1]} - {w[2]}",
                        "Candidate": w[4],
                        "Party": w[0],
                        "Front": w[0],
                        "Votes": w[5],
                        "Year": 2020
                    })

            except Exception as e:
                print(f"      Error: {e}")

            time.sleep(0.1) # Be polite

    except Exception as e:
        print(f"Global Error: {e}")

    # --- 3. SAVE TO CSV ---
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING COMPLETE ---")
        print(df.head())
        filename = "Malappuram_Master_Dataset.csv"
        df.to_csv(filename, index=False)
        print(f"Saved {len(df)} rows to {filename}")
    else:
        print("No data collected.")

if __name__ == "__main__":
    scrape_kerala_final()

--- Starting Final Scraper (Target: Malappuram) ---
1. Session initialized.
2. Fetching Panchayat List...
   [ERROR] List is empty.
   The server might have reverted to default view.


In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def scrape_kerala_robust():
    print("--- Starting Robust Scraper (Hunter-Seeker Mode) ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. INIT SESSION
    try:
        session.get(URL_HOME, verify=False)
        print("1. Session initialized.")
    except:
        pass

    # CONFIG
    DISTRICT_CODE = "D10001" # Malappuram

    # --- STEP 1: HUNT FOR THE LIST ---
    print(f"2. Hunting for Panchayat List (Dist: {DISTRICT_CODE})...")

    # We try these keys in order. 'M' worked once, so it's first.
    # 'S' (Status) and 'L' (List) are fallbacks.
    candidate_keys = ["M", "S", "L", "W", "O"]
    local_bodies = []
    found_key = None

    for key in candidate_keys:
        payload_list = {
            "_p": "dv",
            "_l": "P",
            "_d": DISTRICT_CODE,
            "_s": key
        }

        try:
            resp = session.post(URL_LIST, data=payload_list, verify=False)
            data = resp.json()

            # Check for ANY list > 10 items
            temp_list = []
            if "mdata" in data:
                if isinstance(data["mdata"], list): temp_list = data["mdata"]
                elif isinstance(data["mdata"], dict):
                     for k, v in data["mdata"].items():
                         if isinstance(v, list): temp_list = v; break

            if not temp_list and "summary" in data and isinstance(data["summary"], list):
                temp_list = data["summary"]

            # DID WE FIND IT?
            if len(temp_list) > 10:
                print(f"   -> MATCH! Key '{key}' returned {len(temp_list)} items.")
                local_bodies = temp_list
                found_key = key
                break
            else:
                print(f"   -> Key '{key}' failed (returned {len(temp_list) if temp_list else 0} items).")

        except Exception as e:
            print(f"   -> Key '{key}' error: {e}")

    if not local_bodies:
        print("[CRITICAL FAILURE] Could not find the Panchayat list with any key.")
        return

    # --- STEP 2: SCRAPE DETAILS ---
    print(f"3. Scraping {len(local_bodies)} Panchayats...")

    for item in local_bodies:
        # ID Extraction Logic
        lb_id = None
        lb_name = "Unknown"

        # Smart search for "G1xxxx" ID
        for col in item:
            s_col = str(col)
            if s_col.startswith("G1"):
                lb_id = s_col
            elif len(s_col) > 3 and not s_col.startswith("G") and not s_col.isdigit():
                lb_name = s_col

        if not lb_id: lb_id = item[0] # Fallback
        if lb_name == "Unknown": lb_name = item[1]

        print(f"   -> Processing: {lb_name} ({lb_id})")

        # PAYLOAD FOR DETAILS
        # We assume _s="L" is constant for details (as per your earlier logs)
        payload_details = {
            "_p": "wv",
            "_w": lb_id,
            "_t": "P",
            "_s": "L"
        }

        try:
            resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)
            det_json = resp_det.json()

            # Extract Wards
            wards = []
            if "summary" in det_json and len(det_json["summary"]) > 0:
                wards = det_json["summary"]
            elif "mdata" in det_json:
                if isinstance(det_json["mdata"], list): wards = det_json["mdata"]
                elif isinstance(det_json["mdata"], dict):
                     for k,v in det_json["mdata"].items():
                         if isinstance(v, list): wards = v; break

            # Append Data
            for w in wards:
                master_data.append({
                    "District": "Malappuram",
                    "Local Body": lb_name,
                    "Type": "Grama Panchayat",
                    "WardName": f"{w[1]} - {w[2]}",
                    "Candidate": w[4],
                    "Party": w[0],
                    "Front": w[0],
                    "Votes": w[5],
                    "Year": 2020
                })

        except Exception as e:
            print(f"      Error: {e}")

        time.sleep(0.1)

    # --- 3. SAVE ---
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING SUCCESS ---")
        print(df.head())
        df.to_csv("Malappuram_Robust_Dataset.csv", index=False)
        print(f"Saved {len(df)} rows.")
    else:
        print("No detail data collected.")

if __name__ == "__main__":
    scrape_kerala_robust()

--- Starting Robust Scraper (Hunter-Seeker Mode) ---
1. Session initialized.
2. Hunting for Panchayat List (Dist: D10001)...
   -> Key 'M' failed (returned 5 items).
   -> Key 'S' error: Expecting value: line 8 column 1 (char 7)
   -> Key 'L' failed (returned 5 items).
   -> Key 'W' error: Expecting value: line 8 column 1 (char 7)
   -> Key 'O' error: Expecting value: line 8 column 1 (char 7)
[CRITICAL FAILURE] Could not find the Panchayat list with any key.


In [ ]:
import requests
import pandas as pd
import time
import urllib3

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def scrape_by_brute_force():
    print("--- Starting Direct Scraper (Bypassing List) ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # CONFIGURATION
    DISTRICT_NAME = "Malappuram"
    DISTRICT_PREFIX = "G10" # G + District Number (10 for Malappuram)
    TARGET_YEAR = 2020

    # Range of IDs to check (Malappuram likely has ~100-120 panchayats)
    # We check 1 to 130 to be safe.
    print(f"Checking IDs {DISTRICT_PREFIX}001 to {DISTRICT_PREFIX}130...")

    for i in range(1, 131):
        # Generate ID: G10001, G10002, ... G10130
        lb_id = f"{DISTRICT_PREFIX}{i:03d}"

        # Payload (The one we know works!)
        payload = {
            "_p": "wv",
            "_w": lb_id,
            "_t": "P",
            "_s": "L",
            "_y": TARGET_YEAR # Try adding year. If it fails, remove this line.
        }

        try:
            resp = session.post(URL_DETAILS, data=payload, verify=False)

            # Check if valid
            if resp.status_code != 200:
                print(f"   [{lb_id}] Failed (Status {resp.status_code})")
                continue

            data = resp.json()

            # Find the wards list
            wards = []
            if "summary" in data and len(data["summary"]) > 0:
                wards = data["summary"]
            elif "mdata" in data and isinstance(data["mdata"], list):
                wards = data["mdata"]
            elif "mdata" in data and isinstance(data["mdata"], dict):
                 for k,v in data["mdata"].items():
                     if isinstance(v, list): wards = v; break

            if wards:
                # We found a valid Panchayat!
                # Extract Name from the first row or log it
                # Usually name isn't in the ward row explicitly, but we can just use ID
                # Or sometimes the JSON has a 'header' key with the name.

                # Let's verify we have data
                sample_ward = wards[0]
                # Assuming WardName is index 2, Party index 0

                print(f"   [SUCCESS] {lb_id}: Found {len(wards)} wards.")

                for w in wards:
                    master_data.append({
                        "District": DISTRICT_NAME,
                        "LocalBodyID": lb_id,
                        # "Local Body": "Unknown", # We can merge names later or find name in JSON
                        "Type": "Grama Panchayat",
                        "WardName": f"{w[1]} - {w[2]}",
                        "Candidate": w[4],
                        "Party": w[0],
                        "Front": w[0],
                        "Votes": w[5],
                        "Year": TARGET_YEAR
                    })
            else:
                # Valid JSON but empty (ID might not exist)
                # print(f"   [{lb_id}] No data.") # Uncomment to see misses
                pass

        except Exception as e:
            # print(f"   [{lb_id}] Error: {e}")
            pass

        # Very short sleep to fly through empty IDs
        time.sleep(0.05)

    # SAVE
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING COMPLETE ---")
        print(df.head())
        filename = f"Malappuram_{TARGET_YEAR}_Direct.csv"
        df.to_csv(filename, index=False)
        print(f"Saved {len(df)} rows to {filename}")
    else:
        print("No data found. Try removing '_y' from payload if 2020 fails.")

if __name__ == "__main__":
    scrape_by_brute_force()

--- Starting Direct Scraper (Bypassing List) ---
Checking IDs G10001 to G10130...
   [SUCCESS] G10001: Found 2 wards.
   [SUCCESS] G10002: Found 2 wards.
   [SUCCESS] G10003: Found 2 wards.
   [SUCCESS] G10004: Found 2 wards.
   [SUCCESS] G10005: Found 2 wards.
   [SUCCESS] G10006: Found 2 wards.
   [G10007] Failed (Status 500)
   [SUCCESS] G10008: Found 2 wards.
   [SUCCESS] G10009: Found 2 wards.
   [SUCCESS] G10010: Found 2 wards.
   [SUCCESS] G10011: Found 2 wards.
   [SUCCESS] G10012: Found 2 wards.
   [G10013] Failed (Status 500)
   [SUCCESS] G10014: Found 2 wards.
   [SUCCESS] G10015: Found 2 wards.
   [SUCCESS] G10016: Found 2 wards.
   [SUCCESS] G10017: Found 2 wards.
   [SUCCESS] G10018: Found 2 wards.
   [SUCCESS] G10019: Found 2 wards.
   [SUCCESS] G10020: Found 2 wards.
   [SUCCESS] G10021: Found 2 wards.
   [SUCCESS] G10022: Found 2 wards.
   [SUCCESS] G10023: Found 2 wards.
   [SUCCESS] G10024: Found 2 wards.
   [SUCCESS] G10025: Found 2 wards.
   [SUCCESS] G10026: Found

X-RAY DEBUGGER (G10001)

In [ ]:
import requests
import urllib3
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def xray_debug():
    print("--- X-RAY DEBUGGER (G10001) ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    # Payload WITHOUT year (to avoid defaults)
    payload = {
        "_p": "wv",
        "_w": "G10001",
        "_t": "P",
        "_s": "L"
    }

    try:
        resp = session.post(URL_DETAILS, data=payload, verify=False)
        data = resp.json()

        print("\n[ROOT KEYS]:", list(data.keys()))

        # Check 'summary'
        if "summary" in data:
            print(f"\n[summary] (Length {len(data['summary'])}):")
            print(json.dumps(data["summary"], indent=2, ensure_ascii=False))

        # Check 'mdata'
        if "mdata" in data:
            print(f"\n[mdata] Type: {type(data['mdata'])}")
            if isinstance(data["mdata"], list):
                print(f"Length: {len(data['mdata'])}")
                print(json.dumps(data["mdata"][:2], indent=2, ensure_ascii=False)) # Print first 2 items
            elif isinstance(data["mdata"], dict):
                print("Keys:", data["mdata"].keys())
                if "wards" in data["mdata"]:
                     print(f"Wards found in mdata['wards']! Length: {len(data['mdata']['wards'])}")
                     print(json.dumps(data["mdata"]["wards"][:2], indent=2, ensure_ascii=False))

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    xray_debug()

--- X-RAY DEBUGGER (G10001) ---

[ROOT KEYS]: ['mdata', 'payload', 'summary']

[summary] (Length 2):
[
  [
    "G10001",
    15,
    5,
    0,
    4
  ],
  [
    "G10001",
    15,
    5,
    0,
    4
  ]
]

[mdata] Type: <class 'dict'>
Keys: dict_keys(['rls', 'info', 'ut'])


In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def scrape_kerala_final_fixed():
    print("--- Starting Session-Primed Scraper ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. INIT SESSION (Home)
    try:
        session.get(URL_HOME, verify=False)
        print("1. Session Cookie obtained.")
    except:
        pass

    # CONFIG
    DIST_CODE = "D10001" # Malappuram

    # --- STEP 1: PRIME THE SESSION ---
    # We MUST tell the server which district/year we are in.
    # We call the 'List' endpoint. even if the response is ignored,
    # the server updates $_SESSION['district'] and $_SESSION['year'].
    print("2. Priming Session with District/Year context...")

    payload_prime = {
        "_p": "dv",
        "_l": "P",
        "_d": DIST_CODE,
        "_s": "M"
        # Note: If year is needed, we might need to add it here,
        # but let's stick to what worked in your browser first.
    }

    try:
        resp_prime = session.post(URL_LIST, data=payload_prime, verify=False)
        # We don't care if this returns a list or summary.
        # We just need the cookie to update.
        print("   Context set. Server response length:", len(resp_prime.text))

        # Try to extract valid IDs if possible, otherwise use manual range
        valid_ids = []
        try:
            data_list = resp_prime.json()
            if "mdata" in data_list and isinstance(data_list["mdata"], list):
                valid_ids = [x[0] for x in data_list["mdata"]]
            elif "summary" in data_list and len(data_list["summary"]) > 10:
                valid_ids = [x[0] for x in data_list["summary"]]
        except:
            pass

        if valid_ids:
            print(f"   Success! Found {len(valid_ids)} valid IDs from list.")
        else:
            print("   List parsing failed/empty. Falling back to Brute Force on valid Malappuram IDs.")
            # Known valid range for Malappuram based on your screenshots:
            # G10026, G10072, G10096.
            # We will scan a safe range.
            valid_ids = [f"G10{i:03d}" for i in range(1, 150)]

    except Exception as e:
        print(f"   Priming failed: {e}")
        return

    # --- STEP 2: SCRAPE DETAILS ---
    print(f"3. Scraping Details for {len(valid_ids)} IDs...")

    success_count = 0

    for lb_id in valid_ids:
        # Payload for Details
        payload_details = {
            "_p": "wv",
            "_w": lb_id,
            "_t": "P",
            "_s": "L"
        }

        try:
            resp = session.post(URL_DETAILS, data=payload_details, verify=False)
            data = resp.json()

            # Extract Wards (Look in 'summary' and 'mdata')
            wards = []

            # Logic: If it's the 2-row summary, IGNORE IT. We want the long list.

            potential_lists = []
            if "summary" in data: potential_lists.append(data["summary"])
            if "mdata" in data:
                if isinstance(data["mdata"], list): potential_lists.append(data["mdata"])
                elif isinstance(data["mdata"], dict):
                     for v in data["mdata"].values():
                         if isinstance(v, list): potential_lists.append(v)

            for lst in potential_lists:
                if len(lst) > 5: # Real ward list will have > 5 items (wards)
                    wards = lst
                    break

            if wards:
                # Name is usually not in the row, so we use ID as placeholder or
                # extract if available in a header (not visible in JSON logs so far)
                lb_name = f"Panchayat {lb_id}"

                # Check for Name in the previous "Prime" step if we had it
                # For now, just scrape the data

                for w in wards:
                    # w: [Party, WardNum, WardName, ... Candidate, Votes]
                    if len(w) > 5:
                        master_data.append({
                            "District": "Malappuram",
                            "LocalBodyID": lb_id,
                            "Ward": f"{w[1]} - {w[2]}",
                            "Candidate": w[4],
                            "Party": w[0],
                            "Votes": w[5],
                            "Year": 2020
                        })

                print(f"   [SUCCESS] {lb_id}: {len(wards)} wards scraped.")
                success_count += 1
            else:
                # print(f"   [SKIP] {lb_id}: No detailed ward data (Length {len(wards)})")
                pass

        except Exception as e:
            pass

        time.sleep(0.05) # Fast scan

    # --- SAVE ---
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING COMPLETE ---")
        print(df.head())
        df.to_csv("Malappuram_Final_Fixed.csv", index=False)
        print(f"Saved {len(df)} rows.")
    else:
        print("No data. Session priming might have failed.")

if __name__ == "__main__":
    scrape_kerala_final_fixed()

--- Starting Session-Primed Scraper ---
1. Session Cookie obtained.
2. Priming Session with District/Year context...
   Context set. Server response length: 4668
   List parsing failed/empty. Falling back to Brute Force on valid Malappuram IDs.
3. Scraping Details for 149 IDs...
No data. Session priming might have failed.


Starting 'Walk the Path' Scraper

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def deep_search_for_list(data, min_length=10):
    """Recursively searches JSON for any list longer than min_length."""
    candidates = []

    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) > min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))

    return candidates

def scrape_kerala_walkthrough():
    print("--- Starting 'Walk the Path' Scraper ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # STEP 1: VISIT HOME (Start Session)
    try:
        session.get(URL_HOME, verify=False)
        print("1. Homepage visited (Session started).")
    except:
        pass

    # STEP 2: "CLICK" MALAPPURAM
    # We use exactly the keys from your screenshot: _s="L"
    print("2. Fetching Malappuram List (_s='L')...")

    payload_list = {
        "_p": "dv",
        "_l": "P",
        "_d": "D10001",   # Malappuram
        "_s": "L"         # The key from your screenshot
    }

    try:
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data_list = resp.json()

        # DEEP SEARCH: Find the big list (90+ items) ignoring key names
        found_lists = deep_search_for_list(data_list, min_length=20)

        if not found_lists:
            print("[ERROR] No list found. Server returned keys:", data_list.keys())
            if "mdata" in data_list: print("mdata:", data_list["mdata"])
            return

        # The longest list found is likely our Panchayat list
        local_bodies = max(found_lists, key=len)
        print(f"   -> Success! Found list with {len(local_bodies)} items.")

        # STEP 3: "CLICK" EACH PANCHAYAT
        print("3. Scraping Ward Details...")

        # Loop through the found list
        for item in local_bodies:
            # ID extraction (look for G1xxxx)
            lb_id = None
            lb_name = "Unknown"

            # Robust ID finder
            for col in item:
                s_col = str(col)
                if s_col.startswith("G1"):
                    lb_id = s_col
                elif len(s_col) > 3 and not s_col.startswith("G") and not s_col.isdigit():
                    lb_name = s_col

            if not lb_id: lb_id = item[0]
            if lb_name == "Unknown": lb_name = item[1]

            print(f"   -> Processing: {lb_name} ({lb_id})")

            # Payload for Details (Verified: _s='L')
            payload_details = {
                "_p": "wv",
                "_w": lb_id,
                "_t": "P",
                "_s": "L"
            }

            try:
                resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)
                det_json = resp_det.json()

                # EXTRACT WARDS (Search again for list > 5 items)
                wards_lists = deep_search_for_list(det_json, min_length=5)

                if wards_lists:
                    wards = max(wards_lists, key=len)

                    for w in wards:
                        # Ensure row has enough columns
                        if len(w) >= 6:
                            master_data.append({
                                "District": "Malappuram",
                                "Local Body": lb_name,
                                "Type": "Grama Panchayat",
                                "WardName": f"{w[1]} - {w[2]}", # Adjust indices if needed
                                "Candidate": w[4],
                                "Party": w[0],
                                "Votes": w[5],
                                "Year": 2020
                            })
                else:
                    print(f"      [Warning] No wards found for {lb_name}")

            except Exception as e:
                pass

            time.sleep(0.05) # Speed up

    except Exception as e:
        print(f"Global Error: {e}")

    # SAVE
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SUCCESS! ---")
        print(df.head())
        df.to_csv("Malappuram_Full_Data.csv", index=False)
        print(f"Saved {len(df)} rows.")
    else:
        print("No data collected.")

if __name__ == "__main__":
    scrape_kerala_walkthrough()

--- Starting 'Walk the Path' Scraper ---
1. Homepage visited (Session started).
2. Fetching Malappuram List (_s='L')...
   -> Success! Found list with 94 items.
3. Scraping Ward Details...
   -> Processing: Abdurehman Nagar (G10072)
   -> Processing: Alamkode (G10096)
   -> Processing: Aliparamba (G10043)
   -> Processing: Amarambalam (G10026)
   -> Processing: Anakayam (G10037)
   -> Processing: Angadippuram (G10050)
   -> Processing: Areacode (G10029)
   -> Processing: Athavanad (G10057)
   -> Processing: Chaliyar (G10006)
   -> Processing: Cheacode (G10034)
   -> Processing: Chelembra (G10015)
   -> Processing: Cheriyamundam (G10066)
   -> Processing: Cherukavu (G10008)
   -> Processing: Chokkad (G10023)
   -> Processing: Chungathara (G10005)
   -> Processing: Edakkara (G10003)
   -> Processing: Edapatta (G10028)
   -> Processing: Edappal (G10094)
   -> Processing: Edarikkode (G10078)
   -> Processing: Edavanna (G10036)
   -> Processing: Edayur (G10058)
   -> Processing: Elamkulam (

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def deep_search_for_list(data, min_length=10):
    """Recursively searches JSON for any list longer than min_length."""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) > min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def scrape_kerala_final():
    print("--- Starting Final Corrected Scraper ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. START SESSION
    try:
        session.get(URL_HOME, verify=False)
    except:
        pass

    # 2. GET MALAPPURAM LIST
    print("Fetching Malappuram Panchayat List...")
    payload_list = {
        "_p": "dv", "_l": "P", "_d": "D10001", "_s": "L"
    }

    try:
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data_list = resp.json()

        found_lists = deep_search_for_list(data_list, min_length=20)
        if not found_lists:
            print("[ERROR] List not found.")
            return None

        local_bodies = max(found_lists, key=len)
        print(f"-> Found {len(local_bodies)} Panchayats.")

        # 3. SCRAPE DETAILS
        print("Scraping Ward Details...")

        debug_printed = False # To print just one raw row for verification

        for item in local_bodies:
            # ID Extraction
            lb_id = None
            lb_name = "Unknown"
            for col in item:
                s_col = str(col)
                if s_col.startswith("G1"): lb_id = s_col
                elif len(s_col) > 3 and not s_col.startswith("G") and not s_col.isdigit(): lb_name = s_col

            if not lb_id: lb_id = item[0]
            if lb_name == "Unknown": lb_name = item[1]

            print(f"   Processing: {lb_name} ({lb_id})")

            payload_details = {
                "_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"
            }

            try:
                resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)
                det_json = resp_det.json()

                wards_lists = deep_search_for_list(det_json, min_length=5)

                if wards_lists:
                    wards = max(wards_lists, key=len)

                    # DEBUG: Print the first raw row once to verify columns
                    if not debug_printed and len(wards) > 0:
                        print(f"\n[DEBUG] Raw Row Data: {wards[0]}")
                        print("Mapping Applied: 0=ID, 1=Party, 2=No, 3=Candidate?, 4=Votes, 5=Name\n")
                        debug_printed = True

                    for w in wards:
                        # BASED ON YOUR SCRAMBLED OUTPUT:
                        # w[0] was ID (G10072001)
                        # w[1] was Party (UDF)
                        # w[2] was WardNo (3)
                        # w[3] = CANDIDATE (Assumed, as it's the only slot left)
                        # w[4] was Votes (622)
                        # w[5] was WardName (Valiyaparamba)

                        if len(w) >= 6:
                            master_data.append({
                                "District": "Malappuram",
                                "Local Body": lb_name,
                                "Type": "Grama Panchayat",
                                "WardName": f"{w[2]} - {w[5]}", # Number - Name
                                "Candidate": w[3], # Corrected Index
                                "Party": w[1],
                                "Front": w[1],     # Assuming Front = Party for now
                                "Votes": w[4],     # Corrected Index
                                "Year": 2020
                            })
            except Exception as e:
                pass

            time.sleep(0.05)

    except Exception as e:
        print(f"Global Error: {e}")

    # SAVE AND RETURN
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING SUCCESS ---")
        print(df.head()) # This will show if the columns are right
        df.to_csv("Malappuram_Corrected.csv", index=False)
        print("Saved to 'Malappuram_Corrected.csv'")
        return df
    else:
        return None

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    # We assign the result to 'df' so it persists after the function ends
    df = scrape_kerala_final()

    # Now you can use 'df' or print it
    if df is not None:
        print(f"\nTotal Rows Scraped: {len(df)}")

--- Starting Final Corrected Scraper ---
Fetching Malappuram Panchayat List...
-> Found 94 Panchayats.
Scraping Ward Details...
   Processing: Abdurehman Nagar (G10072)

[DEBUG] Raw Row Data: ['G10072001', 'UDF', 3, 'വേലായുധന്\u200d ചമ്പയിൽ', 622, 'Valiyaparamba', 'Y', 2, '2 - നസീര്\u200d കരിപറമ്പത്ത്', 278]
Mapping Applied: 0=ID, 1=Party, 2=No, 3=Candidate?, 4=Votes, 5=Name

   Processing: Alamkode (G10096)
   Processing: Aliparamba (G10043)
   Processing: Amarambalam (G10026)
   Processing: Anakayam (G10037)
   Processing: Angadippuram (G10050)
   Processing: Areacode (G10029)
   Processing: Athavanad (G10057)
   Processing: Chaliyar (G10006)
   Processing: Cheacode (G10034)
   Processing: Chelembra (G10015)
   Processing: Cheriyamundam (G10066)
   Processing: Cherukavu (G10008)
   Processing: Chokkad (G10023)
   Processing: Chungathara (G10005)
   Processing: Edakkara (G10003)
   Processing: Edapatta (G10028)
   Processing: Edappal (G10094)
   Processing: Edarikkode (G10078)
   Proc

In [ ]:
master_data

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def deep_search_for_list(data, min_length=10):
    """Recursively searches JSON for any list longer than min_length."""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) > min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def scrape_kerala_final():
    print("--- Starting Final Corrected Scraper ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. START SESSION
    try:
        session.get(URL_HOME, verify=False)
    except:
        pass

    # 2. GET MALAPPURAM LIST
    print("Fetching Malappuram Panchayat List...")
    payload_list = {
        "_p": "dv", "_l": "P", "_d": "D10001", "_s": "L"
    }

    try:
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data_list = resp.json()

        found_lists = deep_search_for_list(data_list, min_length=20)
        if not found_lists:
            print("[ERROR] List not found.")
            return None

        local_bodies = max(found_lists, key=len)
        print(f"-> Found {len(local_bodies)} Panchayats.")

        # 3. SCRAPE DETAILS
        print("Scraping Ward Details...")

        debug_printed = False # To print just one raw row for verification

        for item in local_bodies:
            # ID Extraction
            lb_id = None
            lb_name = "Unknown"
            for col in item:
                s_col = str(col)
                if s_col.startswith("G1"): lb_id = s_col
                elif len(s_col) > 3 and not s_col.startswith("G") and not s_col.isdigit(): lb_name = s_col

            if not lb_id: lb_id = item[0]
            if lb_name == "Unknown": lb_name = item[1]

            print(f"   Processing: {lb_name} ({lb_id})")

            payload_details = {
                "_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"
            }

            try:
                resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)
                det_json = resp_det.json()

                wards_lists = deep_search_for_list(det_json, min_length=5)

                if wards_lists:
                    wards = max(wards_lists, key=len)

                    # DEBUG: Print the first raw row once to verify columns
                    if not debug_printed and len(wards) > 0:
                        print(f"\n[DEBUG] Raw Row Data: {wards[0]}")
                        print("Mapping Applied: 0=ID, 1=Party, 2=No, 3=Candidate?, 4=Votes, 5=Name\n")
                        debug_printed = True

                    for w in wards:
                        # BASED ON YOUR SCRAMBLED OUTPUT:
                        # w[0] was ID (G10072001)
                        # w[1] was Party (UDF)
                        # w[2] was WardNo (3)
                        # w[3] = CANDIDATE (Assumed, as it's the only slot left)
                        # w[4] was Votes (622)
                        # w[5] was WardName (Valiyaparamba)

                        if len(w) >= 6:
                            master_data.append({
                                "District": "Malappuram",
                                "Local Body": lb_name,
                                "Type": "Grama Panchayat",
                                "WardName": f"{w[2]} - {w[5]}", # Number - Name
                                "Candidate": w[3], # Corrected Index
                                "Party": w[1],
                                "Front": w[1],     # Assuming Front = Party for now
                                "Votes": w[4],     # Corrected Index
                                "Year": 2025
                            })
            except Exception as e:
                pass

            time.sleep(0.05)

    except Exception as e:
        print(f"Global Error: {e}")

    # SAVE AND RETURN
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING SUCCESS ---")
        print(df.head()) # This will show if the columns are right
        df.to_csv("Malappuram_Corrected.csv", index=False)
        print("Saved to 'Malappuram_Corrected.csv'")
        return df
    else:
        return None

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    # We assign the result to 'df' so it persists after the function ends
    df = scrape_kerala_final()

    # Now you can use 'df' or print it
    if df is not None:
        print(f"\nTotal Rows Scraped: {len(df)}")

Starting Final Scraper (With Malayalam Fix)

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def deep_search_for_list(data, min_length=10):
    """Recursively searches JSON for any list longer than min_length."""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) > min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def scrape_kerala_final():
    print("--- Starting Final Scraper (With Malayalam Fix) ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. START SESSION
    try:
        session.get(URL_HOME, verify=False)
    except:
        pass

    # 2. GET MALAPPURAM LIST
    print("Fetching Malappuram Panchayat List...")
    payload_list = {
        "_p": "dv", "_l": "P", "_d": "D10001", "_s": "L"
    }

    try:
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data_list = resp.json()

        found_lists = deep_search_for_list(data_list, min_length=20)
        if not found_lists:
            print("[ERROR] List not found.")
            return

        local_bodies = max(found_lists, key=len)
        print(f"-> Found {len(local_bodies)} Panchayats.")

        # 3. SCRAPE DETAILS
        print("Scraping Ward Details...")

        for item in local_bodies:
            # ID Extraction
            lb_id = None
            lb_name = "Unknown"
            for col in item:
                s_col = str(col)
                if s_col.startswith("G1"): lb_id = s_col
                elif len(s_col) > 3 and not s_col.startswith("G") and not s_col.isdigit(): lb_name = s_col

            if not lb_id: lb_id = item[0]
            if lb_name == "Unknown": lb_name = item[1]

            print(f"   Processing: {lb_name} ({lb_id})")

            payload_details = {
                "_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"
            }

            try:
                resp_det = session.post(URL_DETAILS, data=payload_details, verify=False)
                det_json = resp_det.json()

                wards_lists = deep_search_for_list(det_json, min_length=5)

                if wards_lists:
                    wards = max(wards_lists, key=len)

                    for w in wards:
                        if len(w) >= 6:
                            master_data.append({
                                "District": "Malappuram",
                                "Local Body": lb_name,
                                "Type": "Grama Panchayat",
                                "WardName": f"{w[2]} - {w[5]}", # Ward No - Ward Name
                                "Candidate": w[3], # Malayalam Name
                                "Party": w[1],     # e.g., UDF (Front)
                                "Front": w[1],     # e.g., UDF (Front)
                                "Votes": w[4],     # Winner Votes
                                "Year": 2020
                            })
            except Exception as e:
                pass

            time.sleep(0.05)

    except Exception as e:
        print(f"Global Error: {e}")

    # 4. SAVE WITH UTF-8-SIG (The Fix!)
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING SUCCESS ---")
        print(df.head())

        # 'utf-8-sig' allows Excel to read Malayalam correctly
        df.to_csv("Malappuram_Final_Fixed.csv", index=False, encoding='utf-8-sig')
        print("Saved to 'Malappuram_Final_Fixed.csv' (Open this in Excel)")
    else:
        print("No data collected.")

if __name__ == "__main__":
    scrape_kerala_final()

--- Starting Final Scraper (With Malayalam Fix) ---
Fetching Malappuram Panchayat List...
-> Found 94 Panchayats.
Scraping Ward Details...
   Processing: Abdurehman Nagar (G10072)
   Processing: Alamkode (G10096)
   Processing: Aliparamba (G10043)
   Processing: Amarambalam (G10026)
   Processing: Anakayam (G10037)
   Processing: Angadippuram (G10050)
   Processing: Areacode (G10029)
   Processing: Athavanad (G10057)
   Processing: Chaliyar (G10006)
   Processing: Cheacode (G10034)
   Processing: Chelembra (G10015)
   Processing: Cheriyamundam (G10066)
   Processing: Cherukavu (G10008)
   Processing: Chokkad (G10023)
   Processing: Chungathara (G10005)
   Processing: Edakkara (G10003)
   Processing: Edapatta (G10028)
   Processing: Edappal (G10094)
   Processing: Edarikkode (G10078)
   Processing: Edavanna (G10036)
   Processing: Edayur (G10058)
   Processing: Elamkulam (G10044)
   Processing: Irumbiliyum (G10059)
   Processing: Kalady (G10095)
   Processing: Kalikavu (G10022)
   Proce

Starting Deep Dive Scraper (Target: 2025 Exact Parties)

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def deep_search_for_list(data, min_length=5):
    """Recursively searches JSON for any list longer than min_length."""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) >= min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def scrape_kerala_2025_deep():
    print("--- Starting Deep Dive Scraper (Target: 2025 Exact Parties) ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. INIT SESSION
    try:
        session.get(URL_HOME, verify=False)
        print("1. Session initialized.")
    except:
        pass

    # 2. GET PANCHAYAT LIST (Malappuram)
    print("2. Fetching Panchayat List for Malappuram (D10001)...")
    payload_list = {
        "_p": "dv", "_l": "P", "_d": "D10001", "_s": "L"
    }

    try:
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data_list = resp.json()
        found_lists = deep_search_for_list(data_list, min_length=50) # Expecting ~90+ bodies

        if not found_lists:
            print("[ERROR] Panchayat list not found.")
            return

        local_bodies = max(found_lists, key=len)
        print(f"   -> Found {len(local_bodies)} Panchayats. Starting Deep Scan...")

        # 3. LOOP PANCHAYATS
        total_panchayats = len(local_bodies)

        for i, item in enumerate(local_bodies):
            # ID Extraction
            lb_id = None
            lb_name = "Unknown"
            for col in item:
                s_col = str(col)
                if s_col.startswith("G1"): lb_id = s_col
                elif len(s_col) > 3 and not s_col.startswith("G") and not s_col.isdigit(): lb_name = s_col

            if not lb_id: lb_id = item[0]
            if lb_name == "Unknown": lb_name = item[1]

            print(f"\n[{i+1}/{total_panchayats}] Scanning: {lb_name} ({lb_id})")

            # 4. GET WARD LIST (Level 3)
            payload_wards = {
                "_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"
            }

            try:
                resp_wards = session.post(URL_DETAILS, data=payload_wards, verify=False)
                wards_json = resp_wards.json()
                ward_lists = deep_search_for_list(wards_json, min_length=5)

                if not ward_lists:
                    print("    -> No wards found.")
                    continue

                wards = max(ward_lists, key=len)
                print(f"    -> Found {len(wards)} Wards. Fetching candidates for each...")

                # 5. DEEP DIVE: LOOP WARDS TO GET CANDIDATES (Level 4)
                for w in wards:
                    # w format from Level 3: [Party(Front), WardNum, WardName, ???, Votes, WinnerName]
                    # We need the Ward ID. It is usually composed of PanchayatID + WardNum
                    # e.g., if Panchayat is G10072 and Ward is 1, WardID is likely G10072001

                    ward_num = str(w[2]).strip() # Assuming Ward Number is at index 2 (e.g. "1")
                    ward_name = str(w[5]).strip() # Assuming Name is at index 5

                    # Construct Ward ID (Format: G10072 + 001)
                    try:
                        ward_id_suffix = f"{int(ward_num):03d}"
                        ward_full_id = f"{lb_id}{ward_id_suffix}"
                    except:
                        # Fallback if ward_num isn't a clean number
                        ward_full_id = f"{lb_id}001"

                    # PAYLOAD FOR CANDIDATES (The one you found!)
                    payload_can = {
                        "_p": "can",        # Candidate View
                        "_w": ward_full_id, # Deep Ward ID
                        "_t": "P",
                        "_s": "L"
                    }

                    try:
                        resp_can = session.post(URL_DETAILS, data=payload_can, verify=False)
                        can_json = resp_can.json()

                        # Find candidate list
                        can_lists = deep_search_for_list(can_json, min_length=1)

                        winner_party = "Unknown"
                        winner_name = "Unknown"
                        winner_votes = 0
                        front_affiliation = w[1] # "UDF" from the previous level

                        if can_lists:
                            candidates = max(can_lists, key=len)

                            # LOGIC: Find the candidate with the MAX votes
                            # Candidate Row Structure (from screenshot):
                            # 0: Party (INC) | 1: ? | 2: Name | ... | Last: Votes

                            best_candidate = None
                            max_v = -1

                            for can in candidates:
                                try:
                                    # Votes usually at the end. Let's look at the last column.
                                    votes = int(can[-1])
                                    if votes > max_v:
                                        max_v = votes
                                        best_candidate = can
                                except:
                                    pass

                            if best_candidate:
                                winner_party = best_candidate[0] # The exact party! (INC, BJP)
                                winner_name = best_candidate[2]  # The name
                                winner_votes = max_v

                        # Use data from Level 3 if Level 4 failed, or merge them
                        if winner_party == "Unknown":
                            winner_party = front_affiliation # Fallback to "UDF"

                        master_data.append({
                            "District": "Malappuram",
                            "Local Body": lb_name,
                            "Type": "Grama Panchayat",
                            "Ward": f"{ward_num} - {ward_name}",
                            "Candidate": winner_name,
                            "Party": winner_party,   # Specific (INC)
                            "Front": front_affiliation, # Broad (UDF)
                            "Votes": winner_votes,
                            "Year": 2025
                        })

                    except Exception as e:
                        # print(f"Level 4 Error: {e}")
                        pass

                    # Be nice to the server!
                    time.sleep(0.05)

            except Exception as e:
                print(f"    Error processing {lb_name}: {e}")

    except Exception as e:
        print(f"Global Error: {e}")

    # SAVE
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING SUCCESS ---")
        print(df.head())
        # UTF-8-SIG for Malayalam support
        df.to_csv("Malappuram_2025_Detailed.csv", index=False, encoding='utf-8-sig')
        print(f"Saved {len(df)} rows to 'Malappuram_2025_Detailed.csv'")
    else:
        print("No data collected.")

if __name__ == "__main__":
    scrape_kerala_2025_deep()

--- Starting Deep Dive Scraper (Target: 2025 Exact Parties) ---
1. Session initialized.
2. Fetching Panchayat List for Malappuram (D10001)...
   -> Found 94 Panchayats. Starting Deep Scan...

[1/94] Scanning: Abdurehman Nagar (G10072)
    -> Found 24 Wards. Fetching candidates for each...

[2/94] Scanning: Alamkode (G10096)
    -> Found 21 Wards. Fetching candidates for each...

[3/94] Scanning: Aliparamba (G10043)
    -> Found 24 Wards. Fetching candidates for each...

[4/94] Scanning: Amarambalam (G10026)
    -> Found 22 Wards. Fetching candidates for each...

[5/94] Scanning: Anakayam (G10037)
    -> Found 24 Wards. Fetching candidates for each...

[6/94] Scanning: Angadippuram (G10050)
    -> Found 24 Wards. Fetching candidates for each...

[7/94] Scanning: Areacode (G10029)
    -> Found 20 Wards. Fetching candidates for each...

[8/94] Scanning: Athavanad (G10057)
    -> Found 24 Wards. Fetching candidates for each...

[9/94] Scanning: Chaliyar (G10006)
    -> Found 16 Wards. Fetc

DIAGNOSTIC: INSPECTING CANDIDATE DATA

In [ ]:
import requests
import urllib3
import json
import pandas as pd

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def inspect_candidate_view():
    print("--- DIAGNOSTIC: INSPECTING CANDIDATE DATA ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    # Target: Abdurehman Nagar (G10072) -> Ward 1 (G10072001)
    # Using the exact keys from your screenshot
    payload = {
        "_p": "can",        # Candidate View
        "_w": "G10072001",  # Ward ID
        "_t": "P",
        "_s": "L"
    }

    try:
        print("Fetching data for Ward G10072001...")
        resp = session.post(URL_DETAILS, data=payload, verify=False)
        data = resp.json()

        # SEARCH FOR THE LIST
        candidates = []

        # Helper to find lists recursively
        def find_lists(d):
            found = []
            if isinstance(d, dict):
                for k, v in d.items():
                    if isinstance(v, list) and len(v) > 0: found.append(v)
                    elif isinstance(v, (dict, list)): found.extend(find_lists(v))
            elif isinstance(d, list):
                found.append(d)
            return found

        all_lists = find_lists(data)

        if all_lists:
            # The candidate list is likely the one with "INC" or "LDF" in it.
            # We print the first row of the longest list found.
            main_list = max(all_lists, key=len)

            print(f"\n[SUCCESS] Found list with {len(main_list)} rows.")
            print("--- RAW ROW SAMPLE (Check this!) ---")
            print(main_list[0])

            # create a sample dataframe to verify
            print("\n--- SAMPLE DATAFRAME ---")
            df = pd.DataFrame(main_list)
            print(df.head())

        else:
            print("[ERROR] No list found in JSON.")
            print("Keys:", data.keys())

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    inspect_candidate_view()

--- DIAGNOSTIC: INSPECTING CANDIDATE DATA ---
Fetching data for Ward G10072001...

[SUCCESS] Found list with 3 rows.
--- RAW ROW SAMPLE (Check this!) ---
['INC', 3, '', 'വേലായുധന്\u200d ചമ്പയിൽ', 622, 1, 'Y']

--- SAMPLE DATAFRAME ---
     0  1 2                    3    4  5  6
0  INC  3     വേലായുധന്‍ ചമ്പയിൽ  622  1  Y
1  IND  2    നസീര്‍ കരിപറമ്പത്ത്  278  2  Y
2  BJP  1               അജികുമാർ   39  0  Y


In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def deep_search_for_list(data, min_length=5):
    """Recursively searches JSON for any list longer than min_length."""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) >= min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def scrape_kerala_2025_final():
    print("--- Starting Final 2025 Deep Scraper ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. INIT SESSION
    try:
        session.get(URL_HOME, verify=False)
    except:
        pass

    # 2. GET PANCHAYAT LIST (Malappuram)
    print("2. Fetching Panchayat List...")
    payload_list = {
        "_p": "dv", "_l": "P", "_d": "D10001", "_s": "L"
    }

    try:
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data_list = resp.json()
        found_lists = deep_search_for_list(data_list, min_length=50)

        if not found_lists:
            print("[ERROR] Panchayat list not found.")
            return

        local_bodies = max(found_lists, key=len)
        print(f"   -> Found {len(local_bodies)} Panchayats.")

        # 3. LOOP PANCHAYATS
        total_panchayats = len(local_bodies)

        for i, item in enumerate(local_bodies):
            # Extract ID and Name
            lb_id = None
            lb_name = "Unknown"
            for col in item:
                s_col = str(col)
                if s_col.startswith("G1"): lb_id = s_col
                elif len(s_col) > 3 and not s_col.startswith("G") and not s_col.isdigit(): lb_name = s_col

            if not lb_id: lb_id = item[0]
            if lb_name == "Unknown": lb_name = item[1]

            print(f"[{i+1}/{total_panchayats}] {lb_name} ({lb_id})")

            # 4. GET WARD LIST (Level 3)
            payload_wards = {
                "_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"
            }

            try:
                resp_wards = session.post(URL_DETAILS, data=payload_wards, verify=False)
                wards_json = resp_wards.json()
                ward_lists = deep_search_for_list(wards_json, min_length=5)

                if not ward_lists:
                    print("    -> No wards found.")
                    continue

                wards = max(ward_lists, key=len)

                # 5. DEEP DIVE: GET CANDIDATES (Level 4)
                for w in wards:
                    # w from Level 3: [Front, WardNum, WardName, ..., Votes, WinnerName]
                    # Indices vary, so we rely on your screenshot for WardNum/Name often being 2 and 5 (or 2 and 3)
                    # Safest bet: find the number

                    ward_num = str(w[2]).strip()
                    ward_name = str(w[5]).strip() # Usually index 5 in Ward View
                    front_affiliation = w[1]      # This is the FRONT (UDF/LDF) from the summary view

                    # Construct Ward ID for Deep Query (G10072 + 001)
                    try:
                        ward_id_suffix = f"{int(ward_num):03d}"
                        ward_full_id = f"{lb_id}{ward_id_suffix}"
                    except:
                        ward_full_id = f"{lb_id}001"

                    # QUERY LEVEL 4 (The Candidate Table)
                    payload_can = {
                        "_p": "can", "_w": ward_full_id, "_t": "P", "_s": "L"
                    }

                    try:
                        resp_can = session.post(URL_DETAILS, data=payload_can, verify=False)
                        can_json = resp_can.json()
                        can_lists = deep_search_for_list(can_json, min_length=1)

                        winner_party = "Unknown"
                        winner_name = "Unknown"
                        winner_votes = 0

                        if can_lists:
                            candidates = max(can_lists, key=len)

                            # LOGIC: Find MAX votes
                            best_candidate = None
                            max_v = -1

                            for can in candidates:
                                try:
                                    # USE DIAGNOSTIC INDICES HERE:
                                    # Index 4 is Votes (e.g., 622)
                                    votes = int(can[4])
                                    if votes > max_v:
                                        max_v = votes
                                        best_candidate = can
                                except:
                                    pass

                            if best_candidate:
                                # USE DIAGNOSTIC INDICES HERE:
                                winner_party = best_candidate[0] # Index 0 = Party (INC)
                                winner_name = best_candidate[3]  # Index 3 = Name (Malayalam)
                                winner_votes = max_v

                        master_data.append({
                            "District": "Malappuram",
                            "Local Body": lb_name,
                            "Type": "Grama Panchayat",
                            "Ward": f"{ward_num} - {ward_name}",
                            "Candidate": winner_name,
                            "Party": winner_party,      # Specific (INC)
                            "Front": front_affiliation, # Broad (UDF) - Carried over
                            "Votes": winner_votes,
                            "Year": 2025
                        })

                    except Exception as e:
                        pass

                    # Minimal sleep to keep it fast but safe
                    time.sleep(0.01)

            except Exception as e:
                pass

    except Exception as e:
        print(f"Global Error: {e}")

    # SAVE AND VERIFY
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SAMPLE DATA (First 5 Rows) ---")
        print(df.head()) # <--- Verification before you celebrate

        filename = "Malappuram_2025_Final.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"\nSaved {len(df)} rows to {filename}")
    else:
        print("No data collected.")

if __name__ == "__main__":
    scrape_kerala_2025_final()

--- Starting Final 2025 Deep Scraper ---
2. Fetching Panchayat List...
   -> Found 94 Panchayats.
[1/94] Abdurehman Nagar (G10072)
[2/94] Alamkode (G10096)
[3/94] Aliparamba (G10043)
[4/94] Amarambalam (G10026)
[5/94] Anakayam (G10037)
[6/94] Angadippuram (G10050)
[7/94] Areacode (G10029)
[8/94] Athavanad (G10057)
[9/94] Chaliyar (G10006)
[10/94] Cheacode (G10034)
[11/94] Chelembra (G10015)
[12/94] Cheriyamundam (G10066)
[13/94] Cherukavu (G10008)
[14/94] Chokkad (G10023)
[15/94] Chungathara (G10005)
[16/94] Edakkara (G10003)
[17/94] Edapatta (G10028)
[18/94] Edappal (G10094)
[19/94] Edarikkode (G10078)
[20/94] Edavanna (G10036)
[21/94] Edayur (G10058)
[22/94] Elamkulam (G10044)
[23/94] Irumbiliyum (G10059)
[24/94] Kalady (G10095)
[25/94] Kalikavu (G10022)
[26/94] Kalpakancheri (G10063)
[27/94] Kannamangalam (G10076)
[28/94] Karulai (G10027)
[29/94] Karuvarakundu (G10024)
[30/94] Kavannur (G10031)
[31/94] Keezhattur (G10046)
[32/94] Keezhuparamba (G10032)
[33/94] Kodur (G10042)
[34/94]

INSPECTING WARD LIST (Level 3)
Fetching Ward List for G10072

In [ ]:
import requests
import urllib3
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def inspect_ward_list():
    print("--- INSPECTING WARD LIST (Level 3) ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    # Target: Abdurehman Nagar (G10072)
    payload = {
        "_p": "wv",
        "_w": "G10072",
        "_t": "P",
        "_s": "L"
    }

    try:
        print("Fetching Ward List for G10072...")
        resp = session.post(URL_DETAILS, data=payload, verify=False)
        data = resp.json()

        # Helper to find lists
        def find_lists(d):
            found = []
            if isinstance(d, dict):
                for k, v in d.items():
                    if isinstance(v, list) and len(v) > 0: found.append(v)
                    elif isinstance(v, (dict, list)): found.extend(find_lists(v))
            elif isinstance(d, list):
                found.append(d)
            return found

        all_lists = find_lists(data)

        if all_lists:
            # The Ward list is the longest one
            main_list = max(all_lists, key=len)

            print(f"\n[SUCCESS] Found list with {len(main_list)} rows.")
            print("--- FIRST 3 ROWS (Look for '001', 'Valiyaparamba') ---")
            for i, row in enumerate(main_list[:3]):
                print(f"Row {i}: {row}")

        else:
            print("[ERROR] No list found.")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    inspect_ward_list()

--- INSPECTING WARD LIST (Level 3) ---
Fetching Ward List for G10072...

[SUCCESS] Found list with 24 rows.
--- FIRST 3 ROWS (Look for '001', 'Valiyaparamba') ---
Row 0: ['G10072001', 'UDF', 3, 'വേലായുധന്\u200d ചമ്പയിൽ', 622, 'Valiyaparamba', 'Y', 2, '2 - നസീര്\u200d കരിപറമ്പത്ത്', 278]
Row 1: ['G10072002', 'UDF', 2, 'ശാദിയ ടീച്ചർ', 748, 'Pukayoor Kunnath', 'Y', 1, '1 - ഭാഗിന വിനയൻ', 422]
Row 2: ['G10072003', 'UDF', 4, 'മുഹമ്മദ് ജാബിർ സി കെ', 516, 'Pukayoor', 'Y', 2, '2 - ഇബ്രാഹിം മൂഴിക്കൽ', 490]


In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def deep_search_for_list(data, min_length=5):
    """Recursively searches JSON for any list longer than min_length."""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) >= min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def scrape_kerala_2025_fixed():
    print("--- Starting Corrected Malappuram 2025 Scraper ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # 1. INIT SESSION
    try:
        session.get(URL_HOME, verify=False)
    except:
        pass

    # 2. GET PANCHAYAT LIST
    print("2. Fetching Panchayat List...")
    payload_list = {
        "_p": "dv", "_l": "P", "_d": "D10001", "_s": "L"
    }

    try:
        resp = session.post(URL_LIST, data=payload_list, verify=False)
        data_list = resp.json()
        found_lists = deep_search_for_list(data_list, min_length=50)

        if not found_lists:
            print("[ERROR] Panchayat list not found.")
            return

        local_bodies = max(found_lists, key=len)
        print(f"   -> Found {len(local_bodies)} Panchayats.")

        # 3. LOOP PANCHAYATS
        total_panchayats = len(local_bodies)

        for i, item in enumerate(local_bodies):
            # Extract Panchayat ID
            lb_id = None
            lb_name = "Unknown"
            for col in item:
                s_col = str(col)
                if s_col.startswith("G1"): lb_id = s_col
                elif len(s_col) > 3 and not s_col.startswith("G") and not s_col.isdigit(): lb_name = s_col

            if not lb_id: lb_id = item[0]
            if lb_name == "Unknown": lb_name = item[1]

            print(f"[{i+1}/{total_panchayats}] {lb_name} ({lb_id})")

            # 4. GET WARD LIST (Level 3)
            payload_wards = {
                "_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"
            }

            try:
                resp_wards = session.post(URL_DETAILS, data=payload_wards, verify=False)
                wards_json = resp_wards.json()
                ward_lists = deep_search_for_list(wards_json, min_length=5)

                if not ward_lists:
                    continue

                wards = max(ward_lists, key=len)

                # 5. LOOP WARDS (Deep Dive)
                for w in wards:
                    # CORRECT MAPPING BASED ON YOUR RAW DATA:
                    # Index 0: Ward ID (G10072001) -> Contains the Ward Number!
                    # Index 1: Front (UDF)
                    # Index 5: Ward Name (Valiyaparamba)

                    ward_full_id = str(w[0])
                    front_affiliation = w[1]
                    ward_name = w[5]

                    # Extract Ward Number (Last 3 digits of ID)
                    ward_num = ward_full_id[-3:]

                    # QUERY CANDIDATE VIEW (Level 4)
                    payload_can = {
                        "_p": "can", "_w": ward_full_id, "_t": "P", "_s": "L"
                    }

                    try:
                        resp_can = session.post(URL_DETAILS, data=payload_can, verify=False)
                        can_json = resp_can.json()
                        can_lists = deep_search_for_list(can_json, min_length=1)

                        winner_party = "Unknown"
                        winner_name = "Unknown"
                        winner_votes = 0

                        if can_lists:
                            candidates = max(can_lists, key=len)

                            # Find winner (Max Votes)
                            best_candidate = None
                            max_v = -1

                            for can in candidates:
                                try:
                                    # Level 4 Mapping (Diagnostic):
                                    # Index 0: Party (INC)
                                    # Index 3: Name (Malayalam)
                                    # Index 4: Votes
                                    votes = int(can[4])
                                    if votes > max_v:
                                        max_v = votes
                                        best_candidate = can
                                except:
                                    pass

                            if best_candidate:
                                winner_party = best_candidate[0] # Specific Party (INC)
                                winner_name = best_candidate[3]  # Name
                                winner_votes = max_v

                        # Fallback for Party if Level 4 failed
                        if winner_party == "Unknown": winner_party = front_affiliation

                        master_data.append({
                            "District": "Malappuram",
                            "Local Body": lb_name,
                            "Type": "Grama Panchayat",
                            "Ward": f"{ward_num} - {ward_name}",
                            "Candidate": winner_name,
                            "Party": winner_party,      # INC/IUML
                            "Front": front_affiliation, # UDF/LDF
                            "Votes": winner_votes,
                            "Year": 2025
                        })

                    except Exception:
                        pass

                    time.sleep(0.01) # Be polite

            except Exception:
                pass

    except Exception as e:
        print(f"Global Error: {e}")

    # SAVE
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SAMPLE DATA ---")
        print(df.head(3))

        filename = "Malappuram_2025_Final_Corrected.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"\nSaved {len(df)} rows to {filename}")
    else:
        print("No data collected.")

if __name__ == "__main__":
    scrape_kerala_2025_fixed()

--- Starting Corrected Malappuram 2025 Scraper ---
2. Fetching Panchayat List...
   -> Found 94 Panchayats.
[1/94] Abdurehman Nagar (G10072)
[2/94] Alamkode (G10096)
[3/94] Aliparamba (G10043)
[4/94] Amarambalam (G10026)
[5/94] Anakayam (G10037)
[6/94] Angadippuram (G10050)
[7/94] Areacode (G10029)
[8/94] Athavanad (G10057)
[9/94] Chaliyar (G10006)
[10/94] Cheacode (G10034)
[11/94] Chelembra (G10015)
[12/94] Cheriyamundam (G10066)
[13/94] Cherukavu (G10008)
[14/94] Chokkad (G10023)
[15/94] Chungathara (G10005)
[16/94] Edakkara (G10003)
[17/94] Edapatta (G10028)
[18/94] Edappal (G10094)
[19/94] Edarikkode (G10078)
[20/94] Edavanna (G10036)
[21/94] Edayur (G10058)
[22/94] Elamkulam (G10044)
[23/94] Irumbiliyum (G10059)
[24/94] Kalady (G10095)
[25/94] Kalikavu (G10022)
[26/94] Kalpakancheri (G10063)
[27/94] Kannamangalam (G10076)
[28/94] Karulai (G10027)
[29/94] Karuvarakundu (G10024)
[30/94] Kavannur (G10031)
[31/94] Keezhattur (G10046)
[32/94] Keezhuparamba (G10032)
[33/94] Kodur (G1004

VERIFYING LOCAL BODY TYPE CODES

In [ ]:
import requests
import urllib3
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def deep_search_for_list(data, min_length=1):
    """Finds list of local bodies inside the JSON"""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) >= min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def verify_type_codes():
    print("--- VERIFYING LOCAL BODY TYPE CODES ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    # Codes we want to verify
    test_codes = {
        "P": "Grama Panchayat (Control)",
        "B": "Block Panchayat (Testing...)",
        "D": "District Panchayat (Testing...)",
        "M": "Municipality (Testing...)",
        "C": "Corporation (Testing...)"
    }

    # We use Malappuram (D10001) for testing
    district_code = "D10001"

    for code, desc in test_codes.items():
        print(f"\nTesting Code: '{code}' ({desc})")

        payload = {
            "_p": "dv",
            "_l": code,  # <--- Changing this
            "_d": district_code,
            "_s": "L"
        }

        try:
            resp = session.post(URL_LIST, data=payload, verify=False)
            data = resp.json()

            # Find the list of bodies
            found_lists = deep_search_for_list(data, min_length=1)

            if found_lists:
                # Get the longest list (usually the main list of bodies)
                main_list = max(found_lists, key=len)
                count = len(main_list)

                # Get first item name (usually Index 1, but we print raw to be sure)
                first_item = main_list[0]

                print(f"   [SUCCESS] Found {count} items.")
                print(f"   First Item Raw: {first_item}")

                # Heuristic Check
                name_sample = str(first_item)
                if "Malappuram" in name_sample and code == "D":
                    print("   -> CONFIRMED: This is District Panchayat data.")
                elif "Vengara" in name_sample and code == "B":
                    # Vengara is a block, so if it appears here, B is likely Block
                    print("   -> LOOKS GOOD: 'Vengara' found in list.")
            else:
                print("   [EMPTY] Server returned valid JSON but no list found.")
                print("   Keys:", data.keys())

        except Exception as e:
            print(f"   [ERROR] Failed: {e}")

if __name__ == "__main__":
    verify_type_codes()

--- VERIFYING LOCAL BODY TYPE CODES ---

Testing Code: 'P' (Grama Panchayat (Control))
   [SUCCESS] Found 94 items.
   First Item Raw: ['G10072', 'Abdurehman Nagar', 24, '13', '22', '0', '0', '2', '0']

Testing Code: 'B' (Block Panchayat (Testing...))
   [SUCCESS] Found 15 items.
   First Item Raw: ['B10109', 'Areacode', 19, '10', '18', '0', '0', '1', '0']

Testing Code: 'D' (District Panchayat (Testing...))
   [SUCCESS] Found 5 items.
   First Item Raw: ['D10001', 'MALAPPURAM', 'LDF', 4, 1, 0, 1, 0, 6]

Testing Code: 'M' (Municipality (Testing...))
   [ERROR] Failed: Expecting value: line 8 column 1 (char 7)

Testing Code: 'C' (Corporation (Testing...))
   [SUCCESS] Found 12 items.
   First Item Raw: ['M10088', 'Kondotty', 41, '21', '31', '2', '0', '8', '0']


Starting MASTER Scraper (Block, Muni, District)

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def deep_search_for_list(data, min_length=1):
    """Recursively searches JSON for any list longer than min_length."""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) >= min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def get_candidates_level_4(session, ward_full_id, front_affiliation):
    """Helper to fetch candidate details (Level 4)"""
    payload_can = {
        "_p": "can", "_w": ward_full_id, "_t": "P", "_s": "L"
    }
    try:
        resp_can = session.post(URL_DETAILS, data=payload_can, verify=False)
        can_json = resp_can.json()
        can_lists = deep_search_for_list(can_json, min_length=1)

        if can_lists:
            candidates = max(can_lists, key=len)

            # Logic: Find MAX votes
            best_candidate = None
            max_v = -1

            for can in candidates:
                try:
                    # Index 4 is Votes (validated in diagnostics)
                    votes = int(can[4])
                    if votes > max_v:
                        max_v = votes
                        best_candidate = can
                except:
                    pass

            if best_candidate:
                # Index 0: Party, Index 3: Name
                return best_candidate[0], best_candidate[3], max_v
    except:
        pass

    return "Unknown", "Unknown", 0

def scrape_kerala_master_2025():
    print("--- Starting MASTER Scraper (Block, Muni, District) ---")
    master_data = []

    session = requests.Session()
    session.headers.update(HEADERS)

    # Init Session
    try:
        session.get(URL_HOME, verify=False)
    except:
        pass

    # --- CONFIGURATION FOR LOOP ---
    # We map the "Server Code" to our "Readable Type"
    # Note: We use 'C' for Municipality because that's where the server put them.
    TASKS = [
        {"code": "B", "type": "Block Panchayat", "desc": "Blocks"},
        {"code": "C", "type": "Municipality", "desc": "Municipalities"},
        {"code": "D", "type": "District Panchayat", "desc": "District Body"} # Special handling
    ]

    for task in TASKS:
        type_code = task['code']
        type_name = task['type']
        print(f"\n>>> STARTING: {type_name} ({task['desc']}) <<<")

        local_bodies = []

        # STEP A: GET LIST OF BODIES
        if type_code == "D":
            # Special Case: District Panchayat is just ONE item (Malappuram)
            # We construct it manually to match the list format
            local_bodies = [['D10001', 'Malappuram District Panchayat']]
            print("   -> Target: Malappuram District Panchayat (D10001)")

        else:
            # Normal Case: Fetch List
            payload_list = {
                "_p": "dv", "_l": type_code, "_d": "D10001", "_s": "L"
            }
            try:
                resp = session.post(URL_LIST, data=payload_list, verify=False)
                data_list = resp.json()
                found_lists = deep_search_for_list(data_list, min_length=5)
                if found_lists:
                    local_bodies = max(found_lists, key=len)
                    print(f"   -> Found {len(local_bodies)} {task['desc']}.")
                else:
                    print(f"   [ERROR] No list found for {type_name}. Skipping.")
                    continue
            except Exception as e:
                print(f"   [ERROR] Network failed for {type_name}: {e}")
                continue

        # STEP B: LOOP THROUGH BODIES
        total_bodies = len(local_bodies)
        for i, item in enumerate(local_bodies):
            # Extract ID/Name
            lb_id = None
            lb_name = "Unknown"

            # Robust ID extraction
            for col in item:
                s_col = str(col)
                # IDs start with B (Block), M (Muni), D (District)
                if len(s_col) > 4 and (s_col.startswith("B1") or s_col.startswith("M1") or s_col.startswith("D1")):
                    lb_id = s_col
                elif len(s_col) > 3 and not any(c.isdigit() for c in s_col):
                    lb_name = s_col

            # Fallback
            if not lb_id: lb_id = item[0]
            if lb_name == "Unknown": lb_name = item[1]

            print(f"   [{i+1}/{total_bodies}] Processing: {lb_name} ({lb_id})")

            # STEP C: GET WARDS/DIVISIONS
            payload_wards = {
                "_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"
            }

            try:
                resp_wards = session.post(URL_DETAILS, data=payload_wards, verify=False)
                wards_json = resp_wards.json()
                ward_lists = deep_search_for_list(wards_json, min_length=3) # Min 3 items

                if not ward_lists:
                    print("      No wards/divisions found.")
                    continue

                wards = max(ward_lists, key=len)

                # STEP D: LOOP WARDS (DEEP DIVE)
                for w in wards:
                    # Index 0: Ward ID, Index 1: Front, Index 5: Name
                    ward_full_id = str(w[0])
                    front_affiliation = w[1]
                    ward_name = w[5]

                    # Extract Ward/Division Number (Last 3 digits)
                    try:
                         ward_num = str(int(ward_full_id[-3:]))
                    except:
                         ward_num = "0"

                    # Get Level 4 Data (Candidate)
                    winner_party, winner_name, winner_votes = get_candidates_level_4(session, ward_full_id, front_affiliation)

                    # Fallback
                    if winner_party == "Unknown": winner_party = front_affiliation

                    master_data.append({
                        "District": "Malappuram",
                        "Local Body Type": type_name, # "Block", "Municipality", etc.
                        "Local Body": lb_name,
                        "Ward": f"{ward_num} - {ward_name}",
                        "Candidate": winner_name,
                        "Party": winner_party,
                        "Front": front_affiliation,
                        "Votes": winner_votes,
                        "Year": 2025
                    })

                    # Tiny sleep
                    time.sleep(0.01)

            except Exception as e:
                # print(f"      Error: {e}")
                pass

            # Save partial progress every 5 bodies
            if (i+1) % 5 == 0:
                pd.DataFrame(master_data).to_csv("Malappuram_Partial_Progress.csv", index=False, encoding='utf-8-sig')

    # FINAL SAVE
    if master_data:
        df = pd.DataFrame(master_data)
        print("\n--- SCRAPING COMPLETE ---")
        print(df.head())
        filename = "Malappuram_All_LocalBodies_2025.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"Saved {len(df)} rows to {filename}")
    else:
        print("No data collected.")

if __name__ == "__main__":
    scrape_kerala_master_2025()

--- Starting MASTER Scraper (Block, Muni, District) ---

>>> STARTING: Block Panchayat (Blocks) <<<
   -> Found 15 Blocks.
   [1/15] Processing: Areacode (B10109)
   [2/15] Processing: Kalikavu (B10108)
   [3/15] Processing: Kondotty (B10106)
   [4/15] Processing: Kuttipuram (B10113)
   [5/15] Processing: Malappuram (B10110)
   [6/15] Processing: Mankada (B10112)
   [7/15] Processing: Nilambur (B10105)
   [8/15] Processing: Perinthalmanna (B10111)
   [9/15] Processing: Perumpadappu (B10119)
   [10/15] Processing: Ponnani (B10118)
   [11/15] Processing: Tanur (B10114)
   [12/15] Processing: Tirur (B10117)
   [13/15] Processing: Tirurangadi (B10116)
   [14/15] Processing: Vengara (B10115)
   [15/15] Processing: Wandoor (B10107)

>>> STARTING: Municipality (Municipalities) <<<
   -> Found 12 Municipalities.
   [1/12] Processing: Kondotty (M10088)
   [2/12] Processing: Kottakkal (M10047)
   [3/12] Processing: Malappuram (M10045)
   [4/12] Processing: Manjeri (M10046)
   [5/12] Processing: 

STARTING UNIVERSAL KERALA ELECTION SCRAPER 2025

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json
import os

# --- 1. CONFIGURATION ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# API ENDPOINTS
URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

# DISTRICT MAP (01 to 14)
DISTRICTS = {
    "01": "Thiruvananthapuram",
    "02": "Kollam",
    "03": "Pathanamthitta",
    "04": "Alappuzha",
    "05": "Kottayam",
    "06": "Idukki",
    "07": "Ernakulam",
    "08": "Thrissur",
    "09": "Palakkad",
    "10": "Malappuram",
    "11": "Kozhikode",
    "12": "Wayanad",
    "13": "Kannur",
    "14": "Kasaragod"
}

# BODY TYPES TO SCAN
# Note: 'C' covers both Municipalities and Corporations in the server backend.
BODY_TYPES = [
    {"code": "P", "name": "Grama Panchayat"},
    {"code": "B", "name": "Block Panchayat"},
    {"code": "D", "name": "District Panchayat"},
    {"code": "C", "name": "Municipality/Corporation"}
]

# --- 2. HELPER FUNCTIONS ---

def deep_search_for_list(data, min_length=1):
    """Recursively searches JSON for any list longer than min_length."""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) >= min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def get_candidates_level_4(session, ward_full_id):
    """
    Fetches the specific Candidate List (Level 4) to get exact Party names.
    Returns: (Party, Name, Votes)
    """
    payload_can = {
        "_p": "can", "_w": ward_full_id, "_t": "P", "_s": "L"
    }
    try:
        resp_can = session.post(URL_DETAILS, data=payload_can, verify=False)
        can_json = resp_can.json()
        can_lists = deep_search_for_list(can_json, min_length=1)

        if can_lists:
            candidates = max(can_lists, key=len)

            # Logic: Find candidate with MAX votes
            best_candidate = None
            max_v = -1

            for can in candidates:
                try:
                    # SCHEMA (Verified): Index 0=Party, 3=Name, 4=Votes
                    votes = int(can[4])
                    if votes > max_v:
                        max_v = votes
                        best_candidate = can
                except:
                    pass

            if best_candidate:
                return best_candidate[0], best_candidate[3], max_v

    except Exception:
        pass

    return "Unknown", "Unknown", 0

# --- 3. MAIN SCRAPER ---

def scrape_all_districts_2025():
    print("--- STARTING UNIVERSAL KERALA ELECTION SCRAPER 2025 ---")

    session = requests.Session()
    session.headers.update(HEADERS)

    # Init Session
    try:
        session.get(URL_HOME, verify=False)
        print("1. Session Initialized.")
    except Exception as e:
        print(f"Warning: Session init failed: {e}")

    # --- OUTER LOOP: DISTRICTS ---
    for dist_code, dist_name in DISTRICTS.items():
        print(f"\n==================================================")
        print(f"PROCESSING DISTRICT: {dist_name} ({dist_code})")
        print(f"==================================================")

        district_data = []
        district_id = f"D{dist_code}001" # e.g., D01001, D10001

        # --- INNER LOOP: BODY TYPES ---
        for b_type in BODY_TYPES:
            type_code = b_type['code']
            type_name = b_type['name']

            print(f"\n   >>> Scanning Type: {type_name} ('{type_code}')")

            # 1. Fetch List of Bodies
            payload_list = {
                "_p": "dv", "_l": type_code, "_d": district_id, "_s": "L"
            }

            local_bodies = []
            try:
                resp = session.post(URL_LIST, data=payload_list, verify=False)
                data_list = resp.json()
                found_lists = deep_search_for_list(data_list, min_length=1) # Min 1 for District/Corp

                if found_lists:
                    # Get the longest list found
                    raw_list = max(found_lists, key=len)

                    # DEDUPLICATION LOGIC
                    # The server might return multiple rows for the same body (status summary)
                    # We extract unique IDs.
                    seen_ids = set()
                    for item in raw_list:
                        lb_id = None
                        lb_name = "Unknown"

                        # ID Extraction Strategy
                        for col in item:
                            s_col = str(col)
                            # Look for IDs starting with G, B, D, M, C followed by numbers
                            if len(s_col) > 4 and s_col[0] in ['G','B','D','M','C'] and s_col[1].isdigit():
                                lb_id = s_col
                                break

                        # Fallback: Index 0
                        if not lb_id: lb_id = str(item[0])

                        # Name Extraction: First non-ID string
                        if lb_name == "Unknown":
                            for col in item:
                                if isinstance(col, str) and col != lb_id and len(col) > 3:
                                    lb_name = col
                                    break

                        if lb_id and lb_id not in seen_ids:
                            seen_ids.add(lb_id)
                            local_bodies.append({'id': lb_id, 'name': lb_name})

                    print(f"      -> Found {len(local_bodies)} unique bodies.")
                else:
                    print(f"      -> No bodies found (Empty list).")
                    continue

            except Exception as e:
                print(f"      -> Error fetching list: {e}")
                continue

            # 2. Loop Through Each Body
            for i, body in enumerate(local_bodies):
                lb_id = body['id']
                lb_name = body['name']

                print(f"      [{i+1}/{len(local_bodies)}] {lb_name} ({lb_id})", end="\r")

                # Fetch Wards (Level 3)
                payload_wards = {
                    "_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"
                }

                try:
                    resp_wards = session.post(URL_DETAILS, data=payload_wards, verify=False)
                    wards_json = resp_wards.json()
                    ward_lists = deep_search_for_list(wards_json, min_length=1)

                    if not ward_lists: continue

                    wards = max(ward_lists, key=len)

                    # Loop Wards (Level 4 Deep Dive)
                    for w in wards:
                        # Schema Assumption from previous analysis:
                        # Index 0: Ward Full ID (contains Number)
                        # Index 1: Front Affiliation (UDF/LDF)
                        # Index 5: Ward Name

                        ward_full_id = str(w[0])
                        front_affiliation = str(w[1])

                        # Name might be at index 5 or 2 depending on view, scanning for string
                        ward_name = "Unknown"
                        if len(w) > 5: ward_name = str(w[5])
                        elif len(w) > 2: ward_name = str(w[2])

                        # Extract Ward Number (Last 3 digits of ID)
                        try:
                            ward_num = str(int(ward_full_id[-3:]))
                        except:
                            ward_num = "0"

                        # FETCH CANDIDATE (Level 4)
                        c_party, c_name, c_votes = get_candidates_level_4(session, ward_full_id)

                        # Fallback for Party
                        if c_party == "Unknown": c_party = front_affiliation

                        district_data.append({
                            "District": dist_name,
                            "Local Body Type": type_name,
                            "Local Body ID": lb_id,
                            "Local Body": lb_name,
                            "Ward Number": ward_num,
                            "Ward Name": ward_name,
                            "Candidate": c_name,
                            "Party": c_party,
                            "Front": front_affiliation,
                            "Votes": c_votes,
                            "Year": 2025
                        })

                        time.sleep(0.001) # Micro-sleep to prevent flooding

                except Exception:
                    pass
            print(f"\n      -> Finished {type_name}")

        # --- SAVE DISTRICT DATA ---
        if district_data:
            filename = f"Kerala_{dist_code}_{dist_name}_2025.csv"
            df = pd.DataFrame(district_data)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"\n[SAVED] {filename} with {len(df)} rows.")
        else:
            print(f"\n[SKIPPED] No data for {dist_name}.")

if __name__ == "__main__":
    scrape_all_districts_2025()

--- STARTING UNIVERSAL KERALA ELECTION SCRAPER 2025 ---
1. Session Initialized.

PROCESSING DISTRICT: Thiruvananthapuram (01)

   >>> Scanning Type: Grama Panchayat ('P')
      -> Found 73 unique bodies.

      -> Finished Grama Panchayat

   >>> Scanning Type: Block Panchayat ('B')
      -> Found 11 unique bodies.

      -> Finished Block Panchayat

   >>> Scanning Type: District Panchayat ('D')
      -> Found 1 unique bodies.

      -> Finished District Panchayat

   >>> Scanning Type: Municipality/Corporation ('C')
      -> Found 1 unique bodies.

      -> Finished Municipality/Corporation

[SAVED] Kerala_01_Thiruvananthapuram_2025.csv with 1611 rows.

PROCESSING DISTRICT: Kollam (02)

   >>> Scanning Type: Grama Panchayat ('P')
      -> Found 68 unique bodies.

      -> Finished Grama Panchayat

   >>> Scanning Type: Block Panchayat ('B')
      -> Found 11 unique bodies.

      -> Finished Block Panchayat

   >>> Scanning Type: District Panchayat ('D')
      -> Found 1 unique bodie

Some districts had missing about urban area, fixing those.. ( For districts like Thiruvananthapuram


In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# --- 1. CONFIGURATION ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

# ONLY THE 6 PROBLEMATIC DISTRICTS
DISTRICTS = {
    "01": "Thiruvananthapuram",
    "02": "Kollam",
    "03": "Pathanamthitta",
    "06": "Idukki",
    "12": "Wayanad",
    "14": "Kasaragod"
}

# DUAL SCAN MODE: Check both 'M' and 'C' to catch everything
BODY_TYPES = [
    {"code": "P", "name": "Grama Panchayat"},
    {"code": "B", "name": "Block Panchayat"},
    {"code": "D", "name": "District Panchayat"},
    {"code": "M", "name": "Municipality"},    # Explicitly check M
    {"code": "C", "name": "Corporation"}      # Explicitly check C
]

# --- 2. HELPER FUNCTIONS ---

def deep_search_for_list(data, min_length=1):
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) >= min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def get_candidates_level_4(session, ward_full_id):
    """Fetches Level 4 Candidate Data (Party/Name/Votes)"""
    payload_can = {
        "_p": "can", "_w": ward_full_id, "_t": "P", "_s": "L"
    }
    try:
        resp_can = session.post(URL_DETAILS, data=payload_can, verify=False)
        can_json = resp_can.json()
        can_lists = deep_search_for_list(can_json, min_length=1)

        if can_lists:
            candidates = max(can_lists, key=len)
            best_candidate = None
            max_v = -1

            for can in candidates:
                try:
                    # SCHEMA: Index 0=Party, 3=Name, 4=Votes
                    votes = int(can[4])
                    if votes > max_v:
                        max_v = votes
                        best_candidate = can
                except:
                    pass

            if best_candidate:
                return best_candidate[0], best_candidate[3], max_v
    except:
        pass

    return "Unknown", "Unknown", 0

# --- 3. MAIN SCRAPER ---

def scrape_problem_districts_2025():
    print("--- STARTING TARGETED FIX FOR 6 DISTRICTS ---")

    session = requests.Session()
    session.headers.update(HEADERS)

    try:
        session.get(URL_HOME, verify=False)
    except:
        pass

    for dist_code, dist_name in DISTRICTS.items():
        print(f"\n==================================================")
        print(f"FIXING DISTRICT: {dist_name} ({dist_code})")
        print(f"==================================================")

        district_data = []
        district_id = f"D{dist_code}001"

        # Track processed IDs to avoid duplicates between 'M' and 'C'
        processed_body_ids = set()

        for b_type in BODY_TYPES:
            type_code = b_type['code']
            type_name = b_type['name']

            print(f"\n   >>> Scanning Type: {type_name} ('{type_code}')")

            # 1. Fetch List
            payload_list = {
                "_p": "dv", "_l": type_code, "_d": district_id, "_s": "L"
            }

            local_bodies = []
            try:
                resp = session.post(URL_LIST, data=payload_list, verify=False)
                data_list = resp.json()
                found_lists = deep_search_for_list(data_list, min_length=1)

                if found_lists:
                    raw_list = max(found_lists, key=len)

                    # Extract unique IDs
                    temp_bodies = []
                    for item in raw_list:
                        lb_id = None
                        lb_name = "Unknown"

                        # ID Extraction
                        for col in item:
                            s_col = str(col)
                            if len(s_col) > 4 and s_col[0] in ['G','B','D','M','C'] and s_col[1].isdigit():
                                lb_id = s_col
                                break
                        if not lb_id: lb_id = str(item[0])

                        # Name Extraction
                        if lb_name == "Unknown":
                            for col in item:
                                if isinstance(col, str) and col != lb_id and len(col) > 3:
                                    lb_name = col
                                    break

                        # DEDUPLICATION CHECK
                        # We specifically filter out 'D' IDs in Urban scans
                        if type_code in ['M', 'C'] and lb_id.startswith('D'):
                             continue # Skip District ID leak

                        if lb_id and lb_id not in processed_body_ids:
                            processed_body_ids.add(lb_id)
                            temp_bodies.append({'id': lb_id, 'name': lb_name})

                    local_bodies = temp_bodies
                    print(f"      -> Found {len(local_bodies)} VALID bodies.")
                else:
                    print(f"      -> No bodies found.")
                    continue

            except Exception:
                continue

            # 2. Loop Bodies
            for i, body in enumerate(local_bodies):
                lb_id = body['id']
                lb_name = body['name']

                print(f"      [{i+1}/{len(local_bodies)}] {lb_name} ({lb_id})", end="\r")

                # Fetch Wards
                payload_wards = {
                    "_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"
                }

                try:
                    resp_wards = session.post(URL_DETAILS, data=payload_wards, verify=False)
                    wards_json = resp_wards.json()
                    ward_lists = deep_search_for_list(wards_json, min_length=1)

                    if not ward_lists: continue

                    wards = max(ward_lists, key=len)

                    for w in wards:
                        ward_full_id = str(w[0])
                        front_affiliation = str(w[1])

                        ward_name = "Unknown"
                        if len(w) > 5: ward_name = str(w[5])
                        elif len(w) > 2: ward_name = str(w[2])

                        try:
                            ward_num = str(int(ward_full_id[-3:]))
                        except:
                            ward_num = "0"

                        # Level 4
                        c_party, c_name, c_votes = get_candidates_level_4(session, ward_full_id)

                        if c_party == "Unknown": c_party = front_affiliation

                        district_data.append({
                            "District": dist_name,
                            "Local Body Type": type_name,
                            "Local Body ID": lb_id,
                            "Local Body": lb_name,
                            "Ward Number": ward_num,
                            "Ward Name": ward_name,
                            "Candidate": c_name,
                            "Party": c_party,
                            "Front": front_affiliation,
                            "Votes": c_votes,
                            "Year": 2025
                        })

                        time.sleep(0.001)

                except Exception:
                    pass
            print(f"\n      -> Finished {type_name}")

        # --- SAVE ---
        if district_data:
            filename = f"Kerala_{dist_code}_{dist_name}_2025.csv"
            df = pd.DataFrame(district_data)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"\n[SAVED] {filename} with {len(df)} rows.")

if __name__ == "__main__":
    scrape_problem_districts_2025()

--- STARTING TARGETED FIX FOR 6 DISTRICTS ---

FIXING DISTRICT: Thiruvananthapuram (01)

   >>> Scanning Type: Grama Panchayat ('P')
      -> Found 73 VALID bodies.

      -> Finished Grama Panchayat

   >>> Scanning Type: Block Panchayat ('B')
      -> Found 11 VALID bodies.

      -> Finished Block Panchayat

   >>> Scanning Type: District Panchayat ('D')
      -> Found 1 VALID bodies.

      -> Finished District Panchayat

   >>> Scanning Type: Municipality ('M')

   >>> Scanning Type: Corporation ('C')
      -> Found 0 VALID bodies.

      -> Finished Corporation

[SAVED] Kerala_01_Thiruvananthapuram_2025.csv with 1583 rows.

FIXING DISTRICT: Kollam (02)

   >>> Scanning Type: Grama Panchayat ('P')
      -> Found 68 VALID bodies.


In [ ]:
import requests
import urllib3
import string

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def deep_search_first_list(data):
    """Finds the first list in the JSON response."""
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) > 0: return v
            if isinstance(v, dict):
                res = deep_search_first_list(v)
                if res: return res
    elif isinstance(data, list):
        return data
    return None

def brute_force_find_codes():
    print("--- BRUTE FORCE DISCOVERY FOR THIRUVANANTHAPURAM (D01001) ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    # We test every letter from A to Z
    alphabet = list(string.ascii_uppercase)

    found_something = False

    for code in alphabet:
        payload = {
            "_p": "dv",
            "_l": code,      # <--- Testing this letter
            "_d": "D01001",  # Thiruvananthapuram
            "_s": "L"
        }

        try:
            resp = session.post(URL_LIST, data=payload, verify=False)
            data = resp.json()

            # Look for a list in the response
            result_list = deep_search_first_list(data)

            if result_list:
                count = len(result_list)
                first_item = result_list[0]

                # We only care if it found a list of Local Bodies
                # Check if the first item looks like a Local Body (has ID)
                sample_str = str(first_item)

                print(f"Code '{code}': Found {count} items.")
                print(f"   -> Sample: {sample_str[:100]}...") # Print first 100 chars

                # HIGHLIGHT INTERESTING FINDS
                if "Attingal" in sample_str or "Nedumangad" in sample_str:
                    print("   *** FOUND MUNICIPALITIES! ***")
                if "Thiruvananthapuram" in sample_str and count > 10:
                    # If count is small (1-5), it's likely District Panchayat. Corp usually has many wards, but here we look for Body List.
                    # Actually Corp list is just 1 item usually.
                    pass

                found_something = True
            else:
                # print(f"Code '{code}': [Empty]")
                pass

        except Exception as e:
            print(f"Code '{code}': Error - {e}")

    if not found_something:
        print("\n[FAILURE] No codes returned any lists. The issue might be the '_d' parameter.")

if __name__ == "__main__":
    brute_force_find_codes()

--- BRUTE FORCE DISCOVERY FOR THIRUVANANTHAPURAM (D01001) ---
Code 'A': Error - Expecting value: line 8 column 1 (char 7)
Code 'B': Found 1 items.
   -> Sample: ['Total', 73, 11, 1, 4, 1]...
Code 'C': Found 1 items.
   -> Sample: ['Total', 73, 11, 1, 4, 1]...
Code 'D': Found 1 items.
   -> Sample: ['Total', 73, 11, 1, 4, 1]...
Code 'E': Error - Expecting value: line 8 column 1 (char 7)
Code 'F': Error - Expecting value: line 8 column 1 (char 7)
Code 'G': Error - Expecting value: line 8 column 1 (char 7)
Code 'H': Error - Expecting value: line 8 column 1 (char 7)
Code 'I': Error - Expecting value: line 8 column 1 (char 7)
Code 'J': Error - Expecting value: line 8 column 1 (char 7)
Code 'K': Error - Expecting value: line 8 column 1 (char 7)
Code 'L': Error - Expecting value: line 8 column 1 (char 7)
Code 'M': Error - Expecting value: line 8 column 1 (char 7)
Code 'N': Error - Expecting value: line 8 column 1 (char 7)
Code 'O': Error - Expecting value: line 8 column 1 (char 7)
Code 'P': F

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json

# --- 1. CONFIGURATION ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

# THE 6 PROBLEM DISTRICTS ONLY
DISTRICTS = {
    "01": "Thiruvananthapuram",
    "02": "Kollam",
    "03": "Pathanamthitta",
    "06": "Idukki",
    "12": "Wayanad",
    "14": "Kasaragod"
}

# SCAN TYPES: We focus on Urban Bodies ('C' covers both in these districts usually)
BODY_TYPES = [
    {"code": "C", "name": "Urban Body (Muni/Corp)"}
]

# --- 2. HELPER FUNCTIONS ---

def find_all_lists(data):
    """Recursively finds ALL lists in the JSON, not just the first one."""
    found_lists = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) > 0:
                found_lists.append(v)
            elif isinstance(v, (dict, list)):
                found_lists.extend(find_all_lists(v))
    elif isinstance(data, list):
        found_lists.append(data) # The list itself
        for item in data:
            if isinstance(item, (dict, list)):
                found_lists.extend(find_all_lists(item))
    return found_lists

def get_candidates_level_4(session, ward_full_id):
    payload_can = {"_p": "can", "_w": ward_full_id, "_t": "P", "_s": "L"}
    try:
        resp_can = session.post(URL_DETAILS, data=payload_can, verify=False)
        can_lists = find_all_lists(resp_can.json())
        if can_lists:
            # Candidate list is usually the one with the most columns
            candidates = max(can_lists, key=lambda l: len(l) if len(l)>0 and isinstance(l[0], list) else 0)

            best_candidate = None
            max_v = -1
            for can in candidates:
                if isinstance(can, list) and len(can) > 4:
                    try:
                        votes = int(can[4])
                        if votes > max_v:
                            max_v = votes
                            best_candidate = can
                    except: pass

            if best_candidate:
                return best_candidate[0], best_candidate[3], max_v
    except:
        pass
    return "Unknown", "Unknown", 0

# --- 3. MAIN SCRAPER ---

def scrape_urban_fix_2025():
    print("--- STARTING URBAN FIX FOR 6 DISTRICTS ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    try: session.get(URL_HOME, verify=False)
    except: pass

    for dist_code, dist_name in DISTRICTS.items():
        print(f"\nProcessing: {dist_name} ({dist_code})")

        # Load existing file to append/update
        filename = f"Kerala_{dist_code}_{dist_name}_2025.csv"
        try:
            existing_df = pd.read_csv(filename)
            # Remove bad rows (Urban bodies with District ID 'D...')
            # We keep 'Grama' and 'Block' rows, and 'District' rows (if valid)
            # Valid District rows usually have 'District Panchayat' as type, but ID Dxx001.
            # The BAD rows are 'Municipality' type with 'D' ID.

            mask_bad = (existing_df['Local Body Type'].str.contains("Municipality|Corporation", case=False)) & \
                       (existing_df['Local Body ID'].str.startswith("D"))

            clean_df = existing_df[~mask_bad]
            print(f"   -> Removed {mask_bad.sum()} bad rows. Current count: {len(clean_df)}")
        except:
            print("   -> File not found, starting fresh.")
            clean_df = pd.DataFrame()

        new_urban_data = []
        district_id = f"D{dist_code}001"

        # FETCH LIST
        payload_list = {"_p": "dv", "_l": "C", "_d": district_id, "_s": "L"}

        try:
            resp = session.post(URL_LIST, data=payload_list, verify=False)
            all_lists = find_all_lists(resp.json())

            valid_bodies = []

            # Smart Filter: Look for the list that actually contains Bodies
            for lst in all_lists:
                if not lst: continue
                # Check first item structure
                first_item = lst[0]
                if isinstance(first_item, list):
                    # Row: [ID, Name, ...]
                    # We check if ID matches M... or C...
                    sample_id = str(first_item[0])
                    # Accept M (Muni), C (Corp), or even names that aren't "Total"
                    if sample_id.startswith("M") or sample_id.startswith("C") or (len(first_item)>1 and "Total" not in str(first_item)):
                        valid_bodies = lst
                        break

            if not valid_bodies:
                print("   -> [ERROR] No valid urban list found via 'C'.")
                # Fallback: Try 'M' just in case? No, stick to User input 'C'.
            else:
                print(f"   -> Found {len(valid_bodies)} Urban Bodies!")

                for body in valid_bodies:
                    # Extract ID and Name
                    lb_id = str(body[0])
                    # If ID is just a number (rare), skip or fix.
                    # If ID is "Total", skip.
                    if "Total" in lb_id: continue

                    lb_name = "Unknown"
                    for col in body:
                        if isinstance(col, str) and len(col) > 3 and col != lb_id:
                            lb_name = col
                            break

                    print(f"      Scanning: {lb_name} ({lb_id})")

                    # FETCH WARDS
                    payload_w = {"_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"}
                    try:
                        resp_w = session.post(URL_DETAILS, data=payload_w, verify=False)
                        w_lists = find_all_lists(resp_w.json())
                        if not w_lists: continue

                        wards = max(w_lists, key=len)

                        for w in wards:
                            ward_id = str(w[0])
                            front = str(w[1])
                            w_name = str(w[5]) if len(w)>5 else str(w[2])
                            w_num = ward_id[-3:]

                            party, c_name, votes = get_candidates_level_4(session, ward_id)
                            if party == "Unknown": party = front

                            new_urban_data.append({
                                "District": dist_name,
                                "Local Body Type": "Municipality/Corporation",
                                "Local Body ID": lb_id,
                                "Local Body": lb_name,
                                "Ward Number": w_num,
                                "Ward Name": w_name,
                                "Candidate": c_name,
                                "Party": party,
                                "Front": front,
                                "Votes": votes,
                                "Year": 2025
                            })
                            time.sleep(0.001)
                    except: pass

        except Exception as e:
            print(f"   -> Network Error: {e}")

        # MERGE AND SAVE
        if new_urban_data:
            new_df = pd.DataFrame(new_urban_data)
            final_df = pd.concat([clean_df, new_df], ignore_index=True)
            final_df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"   -> [SAVED] Updated {filename} (Total: {len(final_df)} rows)")
        else:
            print("   -> No new data found to append.")

if __name__ == "__main__":
    scrape_urban_fix_2025()

--- STARTING URBAN FIX FOR 6 DISTRICTS ---

Processing: Thiruvananthapuram (01)
   -> Removed 0 bad rows. Current count: 1583
   -> Found 5 Urban Bodies!
      Scanning: THIRUVANANTHAPURAM (D01001)
      Scanning: THIRUVANANTHAPURAM (D01001)
      Scanning: THIRUVANANTHAPURAM (D01001)
      Scanning: THIRUVANANTHAPURAM (D01001)
      Scanning: THIRUVANANTHAPURAM (D01001)
   -> [SAVED] Updated Kerala_01_Thiruvananthapuram_2025.csv (Total: 1723 rows)

Processing: Kollam (02)
   -> File not found, starting fresh.
   -> Found 5 Urban Bodies!
      Scanning: KOLLAM (D02001)
      Scanning: KOLLAM (D02001)
      Scanning: KOLLAM (D02001)
      Scanning: KOLLAM (D02001)
      Scanning: KOLLAM (D02001)
   -> [SAVED] Updated Kerala_02_Kollam_2025.csv (Total: 135 rows)

Processing: Pathanamthitta (03)
   -> File not found, starting fresh.
   -> Found 5 Urban Bodies!
      Scanning: PATHANAMTHITTA (D03001)
      Scanning: PATHANAMTHITTA (D03001)
      Scanning: PATHANAMTHITTA (D03001)
      Scann

 JSON X-RAY: THIRUVANANTHAPURAM (Code 'C')

In [ ]:
import requests
import urllib3
import json

# 1. SETUP
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

def x_ray_thiruvananthapuram():
    print("--- JSON X-RAY: THIRUVANANTHAPURAM (Code 'C') ---")
    session = requests.Session()
    session.headers.update(HEADERS)

    # The Exact Payload you confirmed from the browser
    payload = {
        "_p": "dv",
        "_l": "C",
        "_d": "D01001",
        "_s": "L"
    }

    try:
        resp = session.post(URL_LIST, data=payload, verify=False)
        data = resp.json()

        print("\n[RESPONSE KEYS]")
        print(data.keys())

        # Check 'mdata' (Master Data) usually holds the list
        if 'mdata' in data:
            print("\n[INSIDE 'mdata']")
            mdata = data['mdata']
            print(f"Type: {type(mdata)}")
            if isinstance(mdata, list):
                print(f"Length: {len(mdata)}")
                if len(mdata) > 0:
                    print(f"First Item: {mdata[0]}")
            elif isinstance(mdata, str):
                print(f"Content (First 500 chars): {mdata[:500]}")

        # Check 'summary'
        if 'summary' in data:
            print("\n[INSIDE 'summary']")
            print(data['summary'])

        # Print Raw text snippet if it's confusing
        print("\n[RAW JSON SNIPPET (First 1000 chars)]")
        print(json.dumps(data, indent=2)[:1000])

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    x_ray_thiruvananthapuram()

--- JSON X-RAY: THIRUVANANTHAPURAM (Code 'C') ---

[RESPONSE KEYS]
dict_keys(['mdata', 'total', 'summary', 'payload'])

[INSIDE 'mdata']
Type: <class 'dict'>

[INSIDE 'summary']
[['D01001', 'THIRUVANANTHAPURAM', 'LDF', 35, 5, 1, 4, 0, 45], ['D01001', 'THIRUVANANTHAPURAM', 'NDA', 6, 0, 0, 0, 1, 7], ['D01001', 'THIRUVANANTHAPURAM', 'OTH', 0, 0, 0, 0, 0, 0], ['D01001', 'THIRUVANANTHAPURAM', 'Tie', 15, 0, 0, 0, 0, 15], ['D01001', 'THIRUVANANTHAPURAM', 'UDF', 25, 6, 0, 0, 0, 31]]

[RAW JSON SNIPPET (First 1000 chars)]
{
  "mdata": {
    "rls": 0,
    "info": "\ud83d\ude4f Thanks \u2013 Team SEC",
    "ut": "19-Dec-25 11:37:31"
  },
  "total": [
    [
      "Total",
      73,
      11,
      1,
      4,
      1
    ]
  ],
  "summary": [
    [
      "D01001",
      "THIRUVANANTHAPURAM",
      "LDF",
      35,
      5,
      1,
      4,
      0,
      45
    ],
    [
      "D01001",
      "THIRUVANANTHAPURAM",
      "NDA",
      6,
      0,
      0,
      0,
      1,
      7
    ],
    [
     

STARTING 'PAYLOAD' KEY EXTRACTION FIX

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json
import os

# --- 1. CONFIGURATION ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

# THE 6 PROBLEM DISTRICTS
DISTRICTS = {
    "01": "Thiruvananthapuram",
    "02": "Kollam",
    "03": "Pathanamthitta",
    "06": "Idukki",
    "12": "Wayanad",
    "14": "Kasaragod"
}

# --- 2. HELPER FUNCTIONS ---

def deep_search_for_list(data, min_length=1):
    """Fallback list finder (recursive)"""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) >= min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def get_candidates_level_4(session, ward_full_id):
    """Fetches Level 4 Candidate Data"""
    payload_can = {"_p": "can", "_w": ward_full_id, "_t": "P", "_s": "L"}
    try:
        resp_can = session.post(URL_DETAILS, data=payload_can, verify=False)
        can_json = resp_can.json()

        # Candidate list is usually in 'payload' too, but let's be safe and search deep
        # or check 'payload' specifically if consistent.
        # Fallback to deep search is safer for Level 4 as it varies less.
        can_lists = deep_search_for_list(can_json, min_length=1)

        if can_lists:
            candidates = max(can_lists, key=len)
            best_candidate = None
            max_v = -1

            for can in candidates:
                try:
                    # Index 4 is Votes
                    votes = int(can[4])
                    if votes > max_v:
                        max_v = votes
                        best_candidate = can
                except: pass

            if best_candidate:
                return best_candidate[0], best_candidate[3], max_v
    except:
        pass
    return "Unknown", "Unknown", 0

# --- 3. MAIN SCRAPER ---

def scrape_payload_fix_2025():
    print("--- STARTING 'PAYLOAD' KEY EXTRACTION FIX ---")

    session = requests.Session()
    session.headers.update(HEADERS)

    try: session.get(URL_HOME, verify=False)
    except: pass

    for dist_code, dist_name in DISTRICTS.items():
        print(f"\n==================================================")
        print(f"FIXING DISTRICT: {dist_name} ({dist_code})")
        print(f"==================================================")

        # 1. CLEANUP OLD FILE
        filename = f"Kerala_{dist_code}_{dist_name}_2025.csv"
        clean_df = pd.DataFrame()

        if os.path.exists(filename):
            try:
                existing_df = pd.read_csv(filename)
                # Remove ANY row that claims to be Municipality/Corp but has a 'D' ID
                # This wipes the slate clean for Urban bodies.
                mask_bad = (existing_df['Local Body Type'].str.contains("Municipality|Corporation", case=False))
                # We essentially re-scrape ALL urban bodies to be safe.
                # So we keep only Grama (P) and Block (B) and valid District (D) rows.

                # Valid District rows: Type='District Panchayat' AND ID starts with 'D'
                # Bad Urban rows: Type='Municipality...'

                clean_df = existing_df[~mask_bad]
                print(f"   -> Removed {mask_bad.sum()} old Urban rows. Preserved {len(clean_df)} Panchayat/Block rows.")
            except:
                print("   -> File read error, starting fresh.")
        else:
            print("   -> File not found, starting fresh.")

        district_data = []
        district_id = f"D{dist_code}001"

        # 2. FETCH THE 'PAYLOAD' LIST
        # We use _l="C" because your browser inspection confirmed it returns the payload
        payload_list = {"_p": "dv", "_l": "C", "_d": district_id, "_s": "L"}

        try:
            resp = session.post(URL_LIST, data=payload_list, verify=False)
            data_json = resp.json()

            # *** THE KEY FIX: Look in 'payload' explicitly ***
            target_list = data_json.get('payload', [])

            if not target_list:
                print("   -> [ERROR] 'payload' key is empty or missing.")
                continue

            print(f"   -> Found {len(target_list)} Urban Bodies in 'payload'!")

            # 3. LOOP THROUGH BODIES
            for body in target_list:
                # Body Format: ['M01003', 'Attingal', 32, ...]
                lb_id = str(body[0])
                lb_name = str(body[1]).strip()

                print(f"      Scanning: {lb_name} ({lb_id})")

                # FETCH WARDS
                payload_w = {"_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"}

                try:
                    resp_w = session.post(URL_DETAILS, data=payload_w, verify=False)
                    # Wards are usually in 'payload' or the main list.
                    # Let's check 'payload' first, fall back to deep search.
                    w_json = resp_w.json()
                    w_list = w_json.get('payload', [])
                    if not w_list:
                        w_lists_deep = deep_search_for_list(w_json, min_length=1)
                        if w_lists_deep:
                             w_list = max(w_lists_deep, key=len)

                    if not w_list: continue

                    for w in w_list:
                        ward_id = str(w[0])
                        front = str(w[1])
                        # Name/Num logic
                        w_name = "Unknown"
                        if len(w) > 5: w_name = str(w[5])
                        elif len(w) > 2: w_name = str(w[2])

                        try:
                            w_num = str(int(ward_id[-3:]))
                        except:
                            w_num = "0"

                        # Level 4 Candidate
                        party, c_name, votes = get_candidates_level_4(session, ward_id)
                        if party == "Unknown": party = front

                        district_data.append({
                            "District": dist_name,
                            "Local Body Type": "Municipality/Corporation",
                            "Local Body ID": lb_id,
                            "Local Body": lb_name,
                            "Ward Number": w_num,
                            "Ward Name": w_name,
                            "Candidate": c_name,
                            "Party": party,
                            "Front": front,
                            "Votes": votes,
                            "Year": 2025
                        })
                        time.sleep(0.001)

                except Exception as e:
                    # print(f"Ward Error: {e}")
                    pass

        except Exception as e:
            print(f"   -> Network Error: {e}")
            continue

        # 4. SAVE AND MERGE
        if district_data:
            new_df = pd.DataFrame(district_data)
            final_df = pd.concat([clean_df, new_df], ignore_index=True)
            final_df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"   -> [SAVED] Updated {filename} (Total: {len(final_df)} rows)")
        else:
            print("   -> No new data collected.")

if __name__ == "__main__":
    scrape_payload_fix_2025()

--- STARTING 'PAYLOAD' KEY EXTRACTION FIX ---

FIXING DISTRICT: Thiruvananthapuram (01)
   -> Removed 140 old Urban rows. Preserved 1583 Panchayat/Block rows.
   -> Found 5 Urban Bodies in 'payload'!
      Scanning: Attingal (M01003)
      Scanning: Nedumangad (M01002)
      Scanning: Neyyattinkara (M01001)
      Scanning: Thiruvananthapuram (C01001)
      Scanning: Varkala (M01004)
   -> [SAVED] Updated Kerala_01_Thiruvananthapuram_2025.csv (Total: 1837 rows)

FIXING DISTRICT: Kollam (02)
   -> Removed 135 old Urban rows. Preserved 0 Panchayat/Block rows.
   -> Found 5 Urban Bodies in 'payload'!
      Scanning: Karunagappally (M02007)
      Scanning: Kollam (C02002)
      Scanning: Kottarakkara (M02087)
      Scanning: Paravoor (M02005)
      Scanning: Punalur (M02006)
   -> [SAVED] Updated Kerala_02_Kollam_2025.csv (Total: 191 rows)

FIXING DISTRICT: Pathanamthitta (03)
   -> Removed 85 old Urban rows. Preserved 0 Panchayat/Block rows.
   -> Found 4 Urban Bodies in 'payload'!
      S

In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json
import os

# --- 1. CONFIGURATION ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

URL_HOME = "https://lbtrend.kerala.gov.in/"
URL_LIST = "https://lbtrend.kerala.gov.in/includes/stateView2_ajax.php"
URL_DETAILS = "https://lbtrend.kerala.gov.in/includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://lbtrend.kerala.gov.in/"
}

# THE 5 REMAINING DISTRICTS (TVM Removed)
DISTRICTS = {
    "02": "Kollam",
    "03": "Pathanamthitta",
    "06": "Idukki",
    "12": "Wayanad",
    "14": "Kasaragod"
}

# --- 2. HELPER FUNCTIONS ---

def deep_search_for_list(data, min_length=1):
    """Recursive search for standard lists (P, B, D)"""
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) >= min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def get_candidates_level_4(session, ward_full_id):
    """Fetches Level 4 Candidate Data"""
    payload_can = {"_p": "can", "_w": ward_full_id, "_t": "P", "_s": "L"}
    try:
        resp_can = session.post(URL_DETAILS, data=payload_can, verify=False)
        can_json = resp_can.json()

        # Candidate list is usually in 'payload' or standard structure
        can_lists = deep_search_for_list(can_json, min_length=1)

        if can_lists:
            candidates = max(can_lists, key=len)
            best_candidate = None
            max_v = -1

            for can in candidates:
                try:
                    # Index 4 is Votes
                    votes = int(can[4])
                    if votes > max_v:
                        max_v = votes
                        best_candidate = can
                except: pass

            if best_candidate:
                return best_candidate[0], best_candidate[3], max_v
    except:
        pass
    return "Unknown", "Unknown", 0

# --- 3. MAIN SCRAPER ---

def scrape_full_rebuild_final_5():
    print("--- STARTING FULL REBUILD FOR REMAINING 5 DISTRICTS ---")

    session = requests.Session()
    session.headers.update(HEADERS)

    try: session.get(URL_HOME, verify=False)
    except: pass

    for dist_code, dist_name in DISTRICTS.items():
        print(f"\n==================================================")
        print(f"REBUILDING: {dist_name} ({dist_code})")
        print(f"==================================================")

        district_data = []
        district_id = f"D{dist_code}001"

        # ---------------------------------------------------------
        # PART A: STANDARD TYPES (Grama, Block, District)
        # ---------------------------------------------------------
        STANDARD_TYPES = [
            {"code": "P", "name": "Grama Panchayat"},
            {"code": "B", "name": "Block Panchayat"},
            {"code": "D", "name": "District Panchayat"}
        ]

        for b_type in STANDARD_TYPES:
            type_code = b_type['code']
            type_name = b_type['name']
            print(f"   >>> Scanning {type_name} ('{type_code}')...")

            payload_list = {"_p": "dv", "_l": type_code, "_d": district_id, "_s": "L"}
            try:
                resp = session.post(URL_LIST, data=payload_list, verify=False)
                data_list = resp.json()
                found_lists = deep_search_for_list(data_list, min_length=1)

                if not found_lists:
                    print("      -> No bodies found.")
                    continue

                raw_list = max(found_lists, key=len)

                # Deduplication logic
                seen_ids = set()
                bodies_to_scan = []
                for item in raw_list:
                    lb_id = None
                    lb_name = "Unknown"
                    for col in item:
                        s_col = str(col)
                        if len(s_col) > 4 and s_col[0] in ['G','B','D'] and s_col[1].isdigit():
                            lb_id = s_col
                            break
                    if not lb_id: lb_id = str(item[0])

                    if lb_name == "Unknown":
                        for col in item:
                            if isinstance(col, str) and col != lb_id and len(col) > 3:
                                lb_name = col
                                break

                    if lb_id and lb_id not in seen_ids:
                        seen_ids.add(lb_id)
                        bodies_to_scan.append({'id': lb_id, 'name': lb_name})

                # Process Bodies
                for i, body in enumerate(bodies_to_scan):
                    payload_w = {"_p": "wv", "_w": body['id'], "_t": "P", "_s": "L"}
                    try:
                        resp_w = session.post(URL_DETAILS, data=payload_w, verify=False)
                        w_lists = deep_search_for_list(resp_w.json(), min_length=1)
                        if not w_lists: continue

                        wards = max(w_lists, key=len)
                        for w in wards:
                            # Extract Ward Data
                            ward_id = str(w[0])
                            front = str(w[1])
                            w_name = str(w[5]) if len(w)>5 else str(w[2])
                            try: w_num = str(int(ward_id[-3:]))
                            except: w_num = "0"

                            # Level 4
                            party, c_name, votes = get_candidates_level_4(session, ward_id)
                            if party == "Unknown": party = front

                            district_data.append({
                                "District": dist_name,
                                "Local Body Type": type_name,
                                "Local Body ID": body['id'],
                                "Local Body": body['name'],
                                "Ward Number": w_num,
                                "Ward Name": w_name,
                                "Candidate": c_name,
                                "Party": party,
                                "Front": front,
                                "Votes": votes,
                                "Year": 2025
                            })
                    except: pass
                print(f"      -> Completed {type_name}")

            except Exception as e:
                print(f"      -> Error: {e}")

        # ---------------------------------------------------------
        # PART B: URBAN BODIES (Muni/Corp) - PAYLOAD METHOD
        # ---------------------------------------------------------
        print(f"   >>> Scanning Urban Bodies (Payload Method)...")
        payload_urban = {"_p": "dv", "_l": "C", "_d": district_id, "_s": "L"}

        try:
            resp = session.post(URL_LIST, data=payload_urban, verify=False)
            u_json = resp.json()
            target_list = u_json.get('payload', [])

            if target_list:
                print(f"      -> Found {len(target_list)} Urban Bodies.")
                for body in target_list:
                    lb_id = str(body[0])
                    lb_name = str(body[1]).strip()
                    if "Total" in lb_id: continue

                    print(f"      Processing: {lb_name} ({lb_id})")

                    # Fetch Wards
                    payload_w = {"_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"}
                    try:
                        resp_w = session.post(URL_DETAILS, data=payload_w, verify=False)
                        w_json = resp_w.json()
                        w_list = w_json.get('payload', [])
                        if not w_list:
                            w_deep = deep_search_for_list(w_json, min_length=1)
                            if w_deep: w_list = max(w_deep, key=len)

                        if w_list:
                            for w in w_list:
                                ward_id = str(w[0])
                                front = str(w[1])
                                w_name = str(w[5]) if len(w)>5 else str(w[2])
                                try: w_num = str(int(ward_id[-3:]))
                                except: w_num = "0"

                                party, c_name, votes = get_candidates_level_4(session, ward_id)
                                if party == "Unknown": party = front

                                district_data.append({
                                    "District": dist_name,
                                    "Local Body Type": "Municipality/Corporation",
                                    "Local Body ID": lb_id,
                                    "Local Body": lb_name,
                                    "Ward Number": w_num,
                                    "Ward Name": w_name,
                                    "Candidate": c_name,
                                    "Party": party,
                                    "Front": front,
                                    "Votes": votes,
                                    "Year": 2025
                                })
                    except: pass
            else:
                print("      -> No Urban Bodies found in payload.")

        except Exception as e:
            print(f"      -> Urban Error: {e}")

        # ---------------------------------------------------------
        # SAVE FILE
        # ---------------------------------------------------------
        if district_data:
            filename = f"Kerala_{dist_code}_{dist_name}_2025.csv"
            df = pd.DataFrame(district_data)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"   -> [SUCCESS] Saved {filename} with {len(df)} rows.")
        else:
            print("   -> [ERROR] No data collected!")

if __name__ == "__main__":
    scrape_full_rebuild_final_5()

--- STARTING FULL REBUILD FOR REMAINING 5 DISTRICTS ---

REBUILDING: Kollam (02)
   >>> Scanning Grama Panchayat ('P')...
      -> Completed Grama Panchayat
   >>> Scanning Block Panchayat ('B')...
      -> Completed Block Panchayat
   >>> Scanning District Panchayat ('D')...
      -> Completed District Panchayat
   >>> Scanning Urban Bodies (Payload Method)...
      -> Found 5 Urban Bodies.
      Processing: Karunagappally (M02007)
      Processing: Kollam (C02002)
      Processing: Kottarakkara (M02087)
      Processing: Paravoor (M02005)
      Processing: Punalur (M02006)
   -> [SUCCESS] Saved Kerala_02_Kollam_2025.csv with 1698 rows.

REBUILDING: Pathanamthitta (03)
   >>> Scanning Grama Panchayat ('P')...
      -> Completed Grama Panchayat
   >>> Scanning Block Panchayat ('B')...
      -> Completed Block Panchayat
   >>> Scanning District Panchayat ('D')...
      -> Completed District Panchayat
   >>> Scanning Urban Bodies (Payload Method)...
      -> Found 4 Urban Bodies.
      P

Combaing all 14 districts

In [ ]:
import pandas as pd
import glob
import os

def combine_kerala_election_data():
    print("--- COMBINING 14 DISTRICT FILES ---")

    # 1. Find all district CSV files
    # Matches patterns like "Kerala_01_Thiruvananthapuram_2025.csv"
    all_files = glob.glob("Kerala_*_2025.csv")

    if not all_files:
        print("[ERROR] No CSV files found! Make sure you are in the correct directory.")
        return

    print(f"Found {len(all_files)} files: {all_files}")

    # 2. Read and Concatenate
    df_list = []
    for filename in all_files:
        try:
            df = pd.read_csv(filename)
            df_list.append(df)
            print(f"   -> Loaded {filename} ({len(df)} rows)")
        except Exception as e:
            print(f"   -> Error reading {filename}: {e}")

    if not df_list:
        return

    master_df = pd.concat(df_list, ignore_index=True)

    # 3. APPLY FIXES (Data Cleaning)
    print("\n--- APPLYING DATA CLEANING ---")

    # Fix 1: Alappuzha "Ala" Grama Panchayat (G04040) which was "Unknown"
    # We check if ID is G04040 and name is Unknown, then replace with 'Ala'
    mask_ala = (master_df['Local Body ID'] == 'G04040') & (master_df['Local Body'] == 'Unknown')
    if mask_ala.any():
        master_df.loc[mask_ala, 'Local Body'] = 'Ala'
        print(f"   -> Fixed {mask_ala.sum()} rows for 'Ala' Grama Panchayat (G04040).")
    else:
        print("   -> 'Ala' fix not needed (already correct or not found).")

    # 4. SAVE MASTER FILE
    output_filename = "Kerala_LocalBodies_Election_2025_MASTER.csv"
    master_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n--- SUCCESS! ---")
    print(f"Combined File Saved: {output_filename}")
    print(f"Total Rows: {len(master_df)}")
    print(f"Total Districts: {master_df['District'].nunique()}")

    return master_df

# Run the function
df_master = combine_kerala_election_data()

--- COMBINING 14 DISTRICT FILES ---
Found 14 files: ['Kerala_03_Pathanamthitta_2025.csv', 'Kerala_04_Alappuzha_2025.csv', 'Kerala_08_Thrissur_2025.csv', 'Kerala_09_Palakkad_2025.csv', 'Kerala_13_Kannur_2025.csv', 'Kerala_01_Thiruvananthapuram_2025.csv', 'Kerala_14_Kasaragod_2025.csv', 'Kerala_05_Kottayam_2025.csv', 'Kerala_11_Kozhikode_2025.csv', 'Kerala_07_Ernakulam_2025.csv', 'Kerala_12_Wayanad_2025.csv', 'Kerala_06_Idukki_2025.csv', 'Kerala_10_Malappuram_2025.csv', 'Kerala_02_Kollam_2025.csv']
   -> Loaded Kerala_03_Pathanamthitta_2025.csv (981 rows)
   -> Loaded Kerala_04_Alappuzha_2025.csv (1666 rows)
   -> Loaded Kerala_08_Thrissur_2025.csv (2204 rows)
   -> Loaded Kerala_09_Palakkad_2025.csv (2116 rows)
   -> Loaded Kerala_13_Kannur_2025.csv (1812 rows)
   -> Loaded Kerala_01_Thiruvananthapuram_2025.csv (1611 rows)
   -> Loaded Kerala_14_Kasaragod_2025.csv (853 rows)
   -> Loaded Kerala_05_Kottayam_2025.csv (1611 rows)
   -> Loaded Kerala_11_Kozhikode_2025.csv (1903 rows)
   -> 

Starting 2020 Election Scaping

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib3

# Setup
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
URL_2020 = "https://www.sec.kerala.gov.in/results/trend2020/views/index.php"

def inspect_2020_structure():
    print(f"--- INSPECTING 2020 SITE STRUCTURE ---")
    print(f"Target: {URL_2020}")

    try:
        # 1. Fetch Homepage
        # We use verify=False because government certs often expire
        resp = requests.get(URL_2020, verify=False)
        print(f"Connection Status: {resp.status_code}")

        soup = BeautifulSoup(resp.text, 'html.parser')

        # 2. Find Dropdowns (Select tags)
        selects = soup.find_all('select')

        if not selects:
            print("[WARNING] No dropdowns found in HTML. They might be loaded via AJAX.")

        for i, sel in enumerate(selects):
            name = sel.get('name') or sel.get('id') or f"Unnamed_{i}"
            options = sel.find_all('option')

            print(f"\n[{i+1}] DROPDOWN FOUND: '{name}'")
            print(f"    Total Options: {len(options)}")

            # Print first 5 options to verify IDs
            print("    Sample Values:")
            for opt in options[:5]:
                val = opt.get('value')
                txt = opt.text.strip()
                if val:
                    print(f"      Code: '{val}' -> Name: '{txt}'")

            # Check specifically for Malappuram
            for opt in options:
                if "Malappuram" in opt.text:
                    print(f"    [KEY INFO] Malappuram Code is: '{opt.get('value')}'")
                    break

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    inspect_2020_structure()

--- INSPECTING 2020 SITE STRUCTURE ---
Target: https://www.sec.kerala.gov.in/results/trend2020/views/index.php
Connection Status: 200
[WARNING] No dropdowns found in HTML. They might be loaded via AJAX.


In [ ]:
import requests
import pandas as pd
import time
import urllib3
import json
import os

# --- 1. CONFIGURATION ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# *** CRITICAL CHANGE: 2020 URLS ***
BASE_URL = "https://www.sec.kerala.gov.in/results/trend2020/"
URL_HOME = BASE_URL + "views/index.php"
URL_LIST = BASE_URL + "includes/stateView2_ajax.php"
URL_DETAILS = BASE_URL + "includes/lb_ajax2.php"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": URL_HOME
}

# DISTRICT MAP (Standard)
DISTRICTS = {
    "01": "Thiruvananthapuram", "02": "Kollam", "03": "Pathanamthitta",
    "04": "Alappuzha", "05": "Kottayam", "06": "Idukki", "07": "Ernakulam",
    "08": "Thrissur", "09": "Palakkad", "10": "Malappuram", "11": "Kozhikode",
    "12": "Wayanad", "13": "Kannur", "14": "Kasaragod"
}

# BODY TYPES (Same as 2025)
BODY_TYPES = [
    {"code": "P", "name": "Grama Panchayat"},
    {"code": "B", "name": "Block Panchayat"},
    {"code": "D", "name": "District Panchayat"},
    {"code": "C", "name": "Municipality/Corporation"}
]

# --- 2. HELPER FUNCTIONS ---

def deep_search_for_list(data, min_length=1):
    candidates = []
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list) and len(v) >= min_length:
                candidates.append(v)
            elif isinstance(v, (dict, list)):
                candidates.extend(deep_search_for_list(v, min_length))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                candidates.extend(deep_search_for_list(item, min_length))
    return candidates

def get_candidates_level_4(session, ward_full_id):
    """Fetches Level 4 Candidate Data (Party/Name/Votes)"""
    payload_can = {
        "_p": "can", "_w": ward_full_id, "_t": "P", "_s": "L"
    }
    try:
        resp_can = session.post(URL_DETAILS, data=payload_can, verify=False)
        can_json = resp_can.json()
        can_lists = deep_search_for_list(can_json, min_length=1)

        if can_lists:
            candidates = max(can_lists, key=len)
            best_candidate = None
            max_v = -1

            for can in candidates:
                try:
                    # SCHEMA CHECK:
                    # In 2025, Index 4 was votes.
                    # If 2020 is exact replica, this holds true.
                    votes = int(can[4])
                    if votes > max_v:
                        max_v = votes
                        best_candidate = can
                except:
                    pass

            if best_candidate:
                # Index 0: Party, Index 3: Name
                return best_candidate[0], best_candidate[3], max_v
    except:
        pass

    return "Unknown", "Unknown", 0

# --- 3. MAIN SCRAPER (2020 EDITION) ---

def scrape_all_districts_2020():
    print("--- STARTING UNIVERSAL KERALA ELECTION SCRAPER 2020 ---")
    print(f"Target Base URL: {BASE_URL}")

    session = requests.Session()
    session.headers.update(HEADERS)

    # Init Session
    try:
        session.get(URL_HOME, verify=False)
        print("1. Session Initialized.")
    except:
        print("Warning: Session init failed.")

    # --- OUTER LOOP: DISTRICTS ---
    for dist_code, dist_name in DISTRICTS.items():
        print(f"\n==================================================")
        print(f"PROCESSING DISTRICT: {dist_name} ({dist_code})")
        print(f"==================================================")

        district_data = []
        district_id = f"D{dist_code}001"

        # --- INNER LOOP: BODY TYPES ---
        for b_type in BODY_TYPES:
            type_code = b_type['code']
            type_name = b_type['name']

            print(f"\n   >>> Scanning Type: {type_name} ('{type_code}')")

            # 1. Fetch List of Bodies
            payload_list = {
                "_p": "dv", "_l": type_code, "_d": district_id, "_s": "L"
            }

            local_bodies = []
            try:
                resp = session.post(URL_LIST, data=payload_list, verify=False)
                # DEBUG: Check if we got redirected (often happens on legacy sites)
                if resp.url != URL_LIST:
                    print("      [ERROR] Redirected to login/home. API might be blocked.")
                    continue

                data_list = resp.json()
                found_lists = deep_search_for_list(data_list, min_length=1)

                if found_lists:
                    raw_list = max(found_lists, key=len)

                    # Deduplication & ID Extraction
                    seen_ids = set()
                    for item in raw_list:
                        lb_id = None
                        lb_name = "Unknown"

                        for col in item:
                            s_col = str(col)
                            if len(s_col) > 4 and s_col[0] in ['G','B','D','M','C'] and s_col[1].isdigit():
                                lb_id = s_col
                                break

                        if not lb_id: lb_id = str(item[0])

                        if lb_name == "Unknown":
                            for col in item:
                                if isinstance(col, str) and col != lb_id and len(col) > 3:
                                    lb_name = col
                                    break

                        if lb_id and lb_id not in seen_ids:
                            seen_ids.add(lb_id)
                            local_bodies.append({'id': lb_id, 'name': lb_name})

                    print(f"      -> Found {len(local_bodies)} unique bodies.")
                else:
                    print(f"      -> No bodies found (Empty list).")
                    continue

            except Exception as e:
                print(f"      -> Error: {e}")
                continue

            # 2. Loop Through Each Body
            for i, body in enumerate(local_bodies):
                lb_id = body['id']
                lb_name = body['name']

                print(f"      [{i+1}/{len(local_bodies)}] {lb_name} ({lb_id})", end="\r")

                # Fetch Wards
                payload_wards = {
                    "_p": "wv", "_w": lb_id, "_t": "P", "_s": "L"
                }

                try:
                    resp_wards = session.post(URL_DETAILS, data=payload_wards, verify=False)
                    wards_json = resp_wards.json()
                    ward_lists = deep_search_for_list(wards_json, min_length=1)

                    if not ward_lists: continue

                    wards = max(ward_lists, key=len)

                    for w in wards:
                        ward_full_id = str(w[0])
                        front_affiliation = str(w[1])

                        # Name/Num Extraction
                        ward_name = "Unknown"
                        if len(w) > 5: ward_name = str(w[5])
                        elif len(w) > 2: ward_name = str(w[2])

                        try:
                            ward_num = str(int(ward_full_id[-3:]))
                        except:
                            ward_num = "0"

                        # Level 4: Candidate
                        c_party, c_name, c_votes = get_candidates_level_4(session, ward_full_id)

                        if c_party == "Unknown": c_party = front_affiliation

                        district_data.append({
                            "District": dist_name,
                            "Local Body Type": type_name,
                            "Local Body ID": lb_id,
                            "Local Body": lb_name,
                            "Ward Number": ward_num,
                            "Ward Name": ward_name,
                            "Candidate": c_name,
                            "Party": c_party,
                            "Front": front_affiliation,
                            "Votes": c_votes,
                            "Year": 2020
                        })

                        time.sleep(0.001)

                except Exception:
                    pass
            print(f"\n      -> Finished {type_name}")

        # --- SAVE ---
        if district_data:
            filename = f"Kerala_{dist_code}_{dist_name}_2020.csv"
            df = pd.DataFrame(district_data)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"\n[SAVED] {filename} with {len(df)} rows.")

if __name__ == "__main__":
    scrape_all_districts_2020()

--- STARTING UNIVERSAL KERALA ELECTION SCRAPER 2020 ---
Target Base URL: https://www.sec.kerala.gov.in/results/trend2020/
1. Session Initialized.

PROCESSING DISTRICT: Thiruvananthapuram (01)

   >>> Scanning Type: Grama Panchayat ('P')
      -> Found 73 unique bodies.

      -> Finished Grama Panchayat

   >>> Scanning Type: Block Panchayat ('B')
      -> Found 11 unique bodies.

      -> Finished Block Panchayat

   >>> Scanning Type: District Panchayat ('D')
      -> Found 1 unique bodies.

      -> Finished District Panchayat

   >>> Scanning Type: Municipality/Corporation ('C')
      -> Found 1 unique bodies.

      -> Finished Municipality/Corporation

[SAVED] Kerala_01_Thiruvananthapuram_2020.csv with 1503 rows.

PROCESSING DISTRICT: Kollam (02)

   >>> Scanning Type: Grama Panchayat ('P')
      -> Found 68 unique bodies.

      -> Finished Grama Panchayat

   >>> Scanning Type: Block Panchayat ('B')
      -> Found 11 unique bodies.

      -> Finished Block Panchayat

   >>> Sca

Combing 2025, 2020 and 2015 results

In [ ]:
import pandas as pd
import glob
import os

def simple_combine_all():
    print("--- UNIVERSAL CSV MERGER ---")

    # 1. Find ALL CSV files in the folder
    all_csvs = glob.glob("*.csv")

    # Exclude the output filename itself if it exists to avoid infinite loops
    output_filename = "Kerala_Election_Complete_Master.csv"
    files_to_merge = [f for f in all_csvs if f != output_filename]

    if not files_to_merge:
        print("[ERROR] No CSV files found in the folder!")
        print("Please make sure you have uploaded the files.")
        return

    print(f"Found {len(files_to_merge)} files to merge:")
    for f in files_to_merge:
        print(f"  - {f}")

    # 2. Read and Merge
    df_list = []
    for filename in files_to_merge:
        try:
            df = pd.read_csv(filename)

            # Optional: Attempt to detect year if missing
            if 'Year' not in df.columns:
                if '2015' in filename: df['Year'] = 2015
                elif '2020' in filename: df['Year'] = 2020
                elif '2025' in filename: df['Year'] = 2025

            df_list.append(df)
            print(f"    > Loaded {len(df)} rows from {filename}")
        except Exception as e:
            print(f"    x Skipped {filename}: {e}")

    # 3. Save Final File
    if df_list:
        master_df = pd.concat(df_list, ignore_index=True)

        # Sort if possible
        if 'Year' in master_df.columns and 'District' in master_df.columns:
            master_df.sort_values(by=['Year', 'District', 'Local Body'], inplace=True)

        master_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

        print("\n" + "="*30)
        print("SUCCESS! MERGE COMPLETE")
        print("="*30)
        print(f"Saved as: {output_filename}")
        print(f"Total Rows: {len(master_df)}")
        if 'Year' in master_df.columns:
            print("Breakdown by Year:")
            print(master_df['Year'].value_counts().to_string())
    else:
        print("Nothing to save.")

simple_combine_all()

--- UNIVERSAL CSV MERGER ---
Found 3 files to merge:
  - Kerala_LocalBodies_Election_2020_MASTER.csv
  - Kerala_LocalBodies_Election_2025_MASTER.csv
  - Kerala_LocalBodies_Election_2015_MASTER.csv
    > Loaded 21783 rows from Kerala_LocalBodies_Election_2020_MASTER.csv
    > Loaded 23531 rows from Kerala_LocalBodies_Election_2025_MASTER.csv
    > Loaded 21823 rows from Kerala_LocalBodies_Election_2015_MASTER.csv

SUCCESS! MERGE COMPLETE
Saved as: Kerala_Election_Complete_Master.csv
Total Rows: 67137
Breakdown by Year:
Year
2025    23531
2015    21823
2020    21783
